# Build Dataset

> conda env create -f environment.yml

Games

In [3]:
# === ESPN NFL results (disambiguated via abbreviations) ===
import json, time, re
from pathlib import Path
import pandas as pd
import requests

YEAR  = 2025
WEEKS = list(range(1, 19))
DATA_DIR = Path("../data"); DATA_DIR.mkdir(parents=True, exist_ok=True)

SESSION = requests.Session()
SESSION.headers.update({
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/120 Safari/537.36")
})

def norm(s: str) -> str:
    return re.sub(r"\s+"," ",str(s or "")).strip()

# 1) Primary mapping: ESPN abbreviation -> your city-style key
ABBR_KEY = {
    # NFC
    "ARI":"Arizona", "ATL":"Atlanta", "CAR":"Carolina", "CHI":"Chicago",
    "DAL":"Dallas", "DET":"Detroit", "GB":"Green Bay", "LAR":"LA Rams",
    "MIN":"Minnesota", "NO":"New Orleans", "NYG":"NY Giants", "PHI":"Philadelphia",
    "SF":"San Francisco", "SEA":"Seattle", "TB":"Tampa Bay", "WAS":"Washington",
    # AFC
    "BAL":"Baltimore", "BUF":"Buffalo", "CIN":"Cincinnati", "CLE":"Cleveland",
    "DEN":"Denver", "HOU":"Houston", "IND":"Indianapolis", "JAX":"Jacksonville",
    "KC":"Kansas City", "LV":"Las Vegas", "LAC":"LA Chargers", "MIA":"Miami",
    "NE":"New England", "NYJ":"NY Jets", "PIT":"Pittsburgh", "TEN":"Tennessee",
}

# 2) Secondary mapping: full team names -> your city-style key (fallback)
TEAM_KEY = {
    # NFC
    "Arizona Cardinals":"Arizona","Arizona":"Arizona",
    "Atlanta Falcons":"Atlanta","Atlanta":"Atlanta",
    "Carolina Panthers":"Carolina","Carolina":"Carolina",
    "Chicago Bears":"Chicago","Chicago":"Chicago",
    "Dallas Cowboys":"Dallas","Dallas":"Dallas",
    "Detroit Lions":"Detroit","Detroit":"Detroit",
    "Green Bay Packers":"Green Bay","Green Bay":"Green Bay",
    "Los Angeles Rams":"LA Rams","L.A. Rams":"LA Rams","LA Rams":"LA Rams","Rams":"LA Rams",
    "Minnesota Vikings":"Minnesota","Minnesota":"Minnesota",
    "New Orleans Saints":"New Orleans","New Orleans":"New Orleans",
    "New York Giants":"NY Giants","NY Giants":"NY Giants","Giants":"NY Giants",
    "Philadelphia Eagles":"Philadelphia","Philadelphia":"Philadelphia",
    "San Francisco 49ers":"San Francisco","San Francisco":"San Francisco","49ers":"San Francisco",
    "Seattle Seahawks":"Seattle","Seattle":"Seattle",
    "Tampa Bay Buccaneers":"Tampa Bay","Tampa Bay":"Tampa Bay","Buccaneers":"Tampa Bay","Bucs":"Tampa Bay",
    "Washington Commanders":"Washington","Washington":"Washington",
    # AFC
    "Baltimore Ravens":"Baltimore","Baltimore":"Baltimore",
    "Buffalo Bills":"Buffalo","Buffalo":"Buffalo",
    "Cincinnati Bengals":"Cincinnati","Cincinnati":"Cincinnati",
    "Cleveland Browns":"Cleveland","Cleveland":"Cleveland",
    "Denver Broncos":"Denver","Denver":"Denver",
    "Houston Texans":"Houston","Houston":"Houston",
    "Indianapolis Colts":"Indianapolis","Indianapolis":"Indianapolis",
    "Jacksonville Jaguars":"Jacksonville","Jacksonville":"Jacksonville","Jaguars":"Jacksonville","Jags":"Jacksonville",
    "Kansas City Chiefs":"Kansas City","Kansas City":"Kansas City",
    "Las Vegas Raiders":"Las Vegas","Las Vegas":"Las Vegas","Raiders":"Las Vegas",
    "Los Angeles Chargers":"LA Chargers","L.A. Chargers":"LA Chargers","LA Chargers":"LA Chargers","Chargers":"LA Chargers",
    "Miami Dolphins":"Miami","Miami":"Miami",
    "New England Patriots":"New England","New England":"New England",
    "New York Jets":"NY Jets","NY Jets":"NY Jets","Jets":"NY Jets",
    "Pittsburgh Steelers":"Pittsburgh","Pittsburgh":"Pittsburgh",
    "Tennessee Titans":"Tennessee","Tennessee":"Tennessee",
}

def to_key_from_abbr(abbr: str, fallback_name: str) -> str:
    abbr = norm(abbr or "")
    if abbr in ABBR_KEY: 
        return ABBR_KEY[abbr]
    # fallback to full displayName mapping
    name = norm(fallback_name or "")
    return TEAM_KEY.get(name, name)

def fetch_week_json(year: int, week: int, cache: bool=True) -> dict:
    cache_path = DATA_DIR / f"espn_scoreboard_year{year}_week{week}.json"
    if cache and cache_path.exists():
        return json.loads(cache_path.read_text(encoding="utf-8", errors="ignore"))
    url = "https://site.api.espn.com/apis/site/v2/sports/football/nfl/scoreboard"
    params = {"year": year, "week": week, "seasontype": 2}
    r = SESSION.get(url, params=params, timeout=25); r.raise_for_status()
    data = r.json()
    if cache:
        cache_path.write_text(json.dumps(data), encoding="utf-8")
    return data

def parse_week_results(js: dict, week: int) -> pd.DataFrame:
    rows = []
    for ev in js.get("events", []):
        comps = ev.get("competitions", [])
        if not comps: 
            continue
        comp = comps[0]
        status = comp.get("status", {}) or ev.get("status", {})
        st_type = (status.get("type") or {}).get("name")
        completed = (status.get("type") or {}).get("completed", False)

        teams = comp.get("competitors", [])
        if len(teams) != 2:
            continue

        home = next((t for t in teams if t.get("homeAway")=="home"), None)
        away = next((t for t in teams if t.get("homeAway")=="away"), None)
        if not home or not away:
            continue

        def extract(tobj):
            team = (tobj.get("team") or {})
            abbr = team.get("abbreviation") or ""
            disp = team.get("displayName") or team.get("shortDisplayName") or team.get("name") or ""
            city_key = to_key_from_abbr(abbr, disp)   # <- disambiguates LA/NY
            score = to_int(tobj.get("score"))
            tid   = team.get("id")
            return city_key, abbr, disp, score, tid

        def to_int(x):
            try: return int(x)
            except: return None

        home_key, home_abbr, home_disp, home_score, home_tid = extract(home)
        away_key, away_abbr, away_disp, away_score, away_tid = extract(away)

        winner_key = None
        for t in teams:
            if t.get("winner") is True:
                tid = (t.get("team") or {}).get("id")
                if tid == home_tid:
                    winner_key = home_key
                elif tid == away_tid:
                    winner_key = away_key
                break

        game_id_api = comp.get("id") or ev.get("id")
        start = ev.get("date") or comp.get("date")
        notes = ";".join([n.get("headline","") for n in comp.get("notes", [])]) if comp.get("notes") else ""

        rows.append({
            "week": week,
            "game_id_api": game_id_api,
            "start_time": start,
            "status_type": st_type,
            "completed": bool(completed),
            "home_team": home_key, "home_abbr": home_abbr, "home_disp": home_disp,
            "away_team": away_key, "away_abbr": away_abbr, "away_disp": away_disp,
            "home_score": home_score, "away_score": away_score,
            "winner": winner_key,
            "notes": notes,
        })
    return pd.DataFrame(rows)

# ---- Build all weeks ----
weekly = []
for w in WEEKS:
    js = fetch_week_json(YEAR, w, cache=True)
    dfw = parse_week_results(js, w)
    if not dfw.empty:
        weekly.append(dfw)
    time.sleep(0.35)

if weekly:
    results_games = (pd.concat(weekly, ignore_index=True)
                       .drop_duplicates(subset=["week","home_team","away_team"])
                       .reset_index(drop=True))
else:
    results_games = pd.DataFrame(columns=[
        "week","game_id_api","start_time","status_type","completed",
        "home_team","home_abbr","home_disp","away_team","away_abbr","away_disp",
        "home_score","away_score","winner","notes"
    ])

# Canonical label/id
results_games["game_label"] = results_games.apply(
    lambda r: f"{r['away_team']} @ {r['home_team']}", axis=1
)
results_games["game_id"] = results_games.apply(
    lambda r: f"{YEAR}_W{int(r['week']):02d}_{r['away_team']}@{r['home_team']}", axis=1
)

# ---- Expand to team-week long (carry abbreviations) ----
home_rows = (results_games
    .rename(columns={"home_team":"team","home_abbr":"team_abbr",
                     "away_team":"opponent","away_abbr":"opponent_abbr"})
    .assign(home_away="H",
            team_score=lambda d: d["home_score"],
            opp_score=lambda d: d["away_score"],
            win=lambda d: (d["completed"]) & (d["winner"]==d["team"]),
            loss=lambda d: (d["completed"]) & (d["winner"].notna()) & (d["winner"]!=d["team"]),
            margin=lambda d: (d["team_score"].fillna(0) - d["opp_score"].fillna(0)))
    [["week","team","team_abbr","opponent","opponent_abbr","home_away",
      "team_score","opp_score","margin","win","loss","completed",
      "game_id_api","game_id","start_time"]])

away_rows = (results_games
    .rename(columns={"away_team":"team","away_abbr":"team_abbr",
                     "home_team":"opponent","home_abbr":"opponent_abbr"})
    .assign(home_away="A",
            team_score=lambda d: d["away_score"],
            opp_score=lambda d: d["home_score"],
            win=lambda d: (d["completed"]) & (d["winner"]==d["team"]),
            loss=lambda d: (d["completed"]) & (d["winner"].notna()) & (d["winner"]!=d["team"]),
            margin=lambda d: (d["team_score"].fillna(0) - d["opp_score"].fillna(0)))
    [["week","team","team_abbr","opponent","opponent_abbr","home_away",
      "team_score","opp_score","margin","win","loss","completed",
      "game_id_api","game_id","start_time"]])

results_teamweek = (pd.concat([home_rows, away_rows], ignore_index=True)
                    .sort_values(["week","team"])
                    .reset_index(drop=True))

# ---- Save ----
p_games = DATA_DIR / f"results_games_{YEAR}.csv"
p_teamw = DATA_DIR / f"results_teamweek_{YEAR}.csv"
results_games.to_csv(p_games, index=False)
results_teamweek.to_csv(p_teamw, index=False)

print(f"Saved game results -> {p_games}  ({len(results_games)} games)")
print(f"Saved team-week results -> {p_teamw}  ({len(results_teamweek)} rows)")
display(results_games.head(10))
display(results_teamweek.head(10))

Saved game results -> ../data/results_games_2025.csv  (272 games)
Saved team-week results -> ../data/results_teamweek_2025.csv  (544 rows)


,week,game_id_api,start_time,status_type,completed,home_team,home_abbr,home_disp,away_team,away_abbr,away_disp,home_score,away_score,winner,notes,game_label,game_id
0,1,401772510,2025-09-05T00:20Z,STATUS_FINAL,True,Philadelphia,PHI,Philadelphia Eagles,Dallas,DAL,Dallas Cowboys,24,20,Philadelphia,,Dallas @ Philadelphia,2025_W01_Dallas@Philadelphia
1,1,401772714,2025-09-06T00:00Z,STATUS_FINAL,True,LA Chargers,LAC,Los Angeles Chargers,Kansas City,KC,Kansas City Chiefs,27,21,LA Chargers,NFL São Paulo Game,Kansas City @ LA Chargers,2025_W01_Kansas City@LA Chargers
2,1,401772830,2025-09-07T17:00Z,STATUS_FINAL,True,Atlanta,ATL,Atlanta Falcons,Tampa Bay,TB,Tampa Bay Buccaneers,20,23,Tampa Bay,,Tampa Bay @ Atlanta,2025_W01_Tampa Bay@Atlanta
3,1,401772829,2025-09-07T17:00Z,STATUS_FINAL,True,Cleveland,CLE,Cleveland Browns,Cincinnati,CIN,Cincinnati Bengals,16,17,Cincinnati,,Cincinnati @ Cleveland,2025_W01_Cincinnati@Cleveland
4,1,401772719,2025-09-07T17:00Z,STATUS_FINAL,True,Indianapolis,IND,Indianapolis Colts,Miami,MIA,Miami Dolphins,33,8,Indianapolis,,Miami @ Indianapolis,2025_W01_Miami@Indianapolis
5,1,401772720,2025-09-07T17:00Z,STATUS_FINAL,True,New England,NE,New England Patriots,Las Vegas,LV,Las Vegas Raiders,13,20,Las Vegas,,Las Vegas @ New England,2025_W01_Las Vegas@New England
6,1,401772718,2025-09-07T17:00Z,STATUS_FINAL,True,New Orleans,NO,New Orleans Saints,Arizona,ARI,Arizona Cardinals,13,20,Arizona,,Arizona @ New Orleans,2025_W01_Arizona@New Orleans
7,1,401772721,2025-09-07T17:00Z,STATUS_FINAL,True,NY Jets,NYJ,New York Jets,Pittsburgh,PIT,Pittsburgh Steelers,32,34,Pittsburgh,,Pittsburgh @ NY Jets,2025_W01_Pittsburgh@NY Jets
8,1,401772827,2025-09-07T17:00Z,STATUS_FINAL,True,Washington,WSH,Washington Commanders,NY Giants,NYG,New York Giants,21,6,Washington,,NY Giants @ Washington,2025_W01_NY Giants@Washington
9,1,401772828,2025-09-07T17:00Z,STATUS_FINAL,True,Jacksonville,JAX,Jacksonville Jaguars,Carolina,CAR,Carolina Panthers,26,10,Jacksonville,,Carolina @ Jacksonville,2025_W01_Carolina@Jacksonville


,week,team,team_abbr,opponent,opponent_abbr,home_away,team_score,opp_score,margin,win,loss,completed,game_id_api,game_id,start_time
0,1,Arizona,ARI,New Orleans,NO,A,20,13,7,True,False,True,401772718,2025_W01_Arizona@New Orleans,2025-09-07T17:00Z
1,1,Atlanta,ATL,Tampa Bay,TB,H,20,23,-3,False,True,True,401772830,2025_W01_Tampa Bay@Atlanta,2025-09-07T17:00Z
2,1,Baltimore,BAL,Buffalo,BUF,A,40,41,-1,False,True,True,401772918,2025_W01_Baltimore@Buffalo,2025-09-08T00:20Z
3,1,Buffalo,BUF,Baltimore,BAL,H,41,40,1,True,False,True,401772918,2025_W01_Baltimore@Buffalo,2025-09-08T00:20Z
4,1,Carolina,CAR,Jacksonville,JAX,A,10,26,-16,False,True,True,401772828,2025_W01_Carolina@Jacksonville,2025-09-07T17:00Z
5,1,Chicago,CHI,Minnesota,MIN,H,24,27,-3,False,True,True,401772810,2025_W01_Minnesota@Chicago,2025-09-09T00:15Z
6,1,Cincinnati,CIN,Cleveland,CLE,A,17,16,1,True,False,True,401772829,2025_W01_Cincinnati@Cleveland,2025-09-07T17:00Z
7,1,Cleveland,CLE,Cincinnati,CIN,H,16,17,-1,False,True,True,401772829,2025_W01_Cincinnati@Cleveland,2025-09-07T17:00Z
8,1,Dallas,DAL,Philadelphia,PHI,A,20,24,-4,False,True,True,401772510,2025_W01_Dallas@Philadelphia,2025-09-05T00:20Z
9,1,Denver,DEN,Tennessee,TEN,H,20,12,8,True,False,True,401772832,2025_W01_Tennessee@Denver,2025-09-07T20:05Z


W-L ratio

In [4]:
# === Compute Win-Loss snapshot per team (up to a given week) ===
import pandas as pd
from pathlib import Path

YEAR = 2025
WEEK = 4   # <- change this to the week you want
DATA = Path("../data")

tw = pd.read_csv(DATA / f"results_teamweek_{YEAR}.csv")

# Aggregate wins/losses up to the chosen week
standings = (tw[tw["week"] <= WEEK]
    .groupby("team")
    .agg(wins=("win","sum"), losses=("loss","sum"))
    .reset_index())

# Add games played and win percentage
standings["games_played"] = standings["wins"] + standings["losses"]
standings["win_pct"] = standings["wins"] / standings["games_played"].replace(0, pd.NA)

# Sort by win %
standings = standings.sort_values(["win_pct","wins"], ascending=[False, False]).reset_index(drop=True)

# Save as its own file
out_path = DATA / f"standings_week{WEEK}_{YEAR}.csv"
standings.to_csv(out_path, index=False)

print(f"Saved snapshot standings for Week {WEEK} -> {out_path}")
display(standings.head(12))

Saved snapshot standings for Week 4 -> ../data/standings_week4_2025.csv


,team,wins,losses,games_played,win_pct
0,Buffalo,3,0,3,1.000000
1,Indianapolis,3,0,3,1.000000
2,LA Chargers,3,0,3,1.000000
3,Philadelphia,3,0,3,1.000000
4,San Francisco,3,0,3,1.000000
5,Tampa Bay,3,0,3,1.000000
6,Seattle,3,1,4,0.750000
7,Cincinnati,2,1,3,0.666667
8,Detroit,2,1,3,0.666667
9,Green Bay,2,1,3,0.666667


### Odds

In [5]:
# === Fetch ALL weeks' moneylines (past + future) -> odds_long_{YEAR}.csv ===
import os, re, time, datetime as dt
from pathlib import Path
import pandas as pd
import requests
import numpy as np

YEAR = 2025
WEEKS = list(range(1, 19))
DATA_DIR = Path("../data"); DATA_DIR.mkdir(parents=True, exist_ok=True)

API_KEY = os.getenv("THE_ODDS_API_KEY") or "a25d13b6734a800828e661ebb91a17ca"

def clean(s: str) -> str:
    return re.sub(r"\s+", " ", str(s or "")).strip()

TEAM_KEY = {
    # NFC
    "Arizona Cardinals":"Arizona","Arizona":"Arizona",
    "Atlanta Falcons":"Atlanta","Atlanta":"Atlanta",
    "Carolina Panthers":"Carolina","Carolina":"Carolina",
    "Chicago Bears":"Chicago","Chicago":"Chicago",
    "Dallas Cowboys":"Dallas","Dallas":"Dallas",
    "Detroit Lions":"Detroit","Detroit":"Detroit",
    "Green Bay Packers":"Green Bay","Green Bay":"Green Bay",
    "Los Angeles Rams":"LA Rams","L.A. Rams":"LA Rams","LA Rams":"LA Rams","Rams":"LA Rams",
    "Minnesota Vikings":"Minnesota","Minnesota":"Minnesota",
    "New Orleans Saints":"New Orleans","New Orleans":"New Orleans",
    "New York Giants":"NY Giants","NY Giants":"NY Giants","Giants":"NY Giants",
    "Philadelphia Eagles":"Philadelphia","Philadelphia":"Philadelphia",
    "San Francisco 49ers":"San Francisco","San Francisco":"San Francisco",
    "Seattle Seahawks":"Seattle","Seattle":"Seattle",
    "Tampa Bay Buccaneers":"Tampa Bay","Tampa Bay":"Tampa Bay",
    "Washington Commanders":"Washington","Washington":"Washington",
    # AFC
    "Baltimore Ravens":"Baltimore","Baltimore":"Baltimore",
    "Buffalo Bills":"Buffalo","Buffalo":"Buffalo",
    "Cincinnati Bengals":"Cincinnati","Cincinnati":"Cincinnati",
    "Cleveland Browns":"Cleveland","Cleveland":"Cleveland",
    "Denver Broncos":"Denver","Denver":"Denver",
    "Houston Texans":"Houston","Houston":"Houston",
    "Indianapolis Colts":"Indianapolis","Indianapolis":"Indianapolis",
    "Jacksonville Jaguars":"Jacksonville","Jacksonville":"Jacksonville",
    "Kansas City Chiefs":"Kansas City","Kansas City":"Kansas City",
    "Las Vegas Raiders":"Las Vegas","Las Vegas":"Las Vegas","Raiders":"Las Vegas",
    "Los Angeles Chargers":"LA Chargers","L.A. Chargers":"LA Chargers","LA Chargers":"LA Chargers","Chargers":"LA Chargers",
    "Miami Dolphins":"Miami","Miami":"Miami",
    "New England Patriots":"New England","New England":"New England",
    "New York Jets":"NY Jets","NY Jets":"NY Jets","Jets":"NY Jets",
    "Pittsburgh Steelers":"Pittsburgh","Pittsburgh":"Pittsburgh",
    "Tennessee Titans":"Tennessee","Tennessee":"Tennessee",
}
def to_key(x: str) -> str:
    x = clean(x).replace("N.Y.","NY").replace("L.A.","LA")
    return TEAM_KEY.get(x, x)

def nfl_week_window(year: int, week: int) -> tuple[dt.datetime, dt.datetime]:
    base = dt.datetime(year, 9, 1, tzinfo=dt.timezone.utc)
    while base.weekday() != 3:  # Thu
        base += dt.timedelta(days=1)
    start = base + dt.timedelta(days=(week-1)*7 - 1)   # Wed before TNF
    end   = start + dt.timedelta(days=8)               # through next Thu AM
    return start, end

SESSION = requests.Session()
SESSION.headers.update({"User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                                       "Chrome/120 Safari/537.36")})

def fetch_week_events(year: int, week: int) -> list:
    """Try odds-history (covers past) then fallback to odds (future)."""
    start, end = nfl_week_window(year, week)
    iso = lambda d: d.isoformat().replace("+00:00","Z")
    common = {
        "regions": "us",
        "markets": "h2h",
        "oddsFormat": "american",
        "apiKey": API_KEY,
        "commenceTimeFrom": iso(start),
        "commenceTimeTo": iso(end),
        "dateFormat": "iso",
        "page": 1,
    }
    # 1) odds-history (past + present snapshots)
    url_hist = "https://api.the-odds-api.com/v4/sports/americanfootball_nfl/odds-history"
    try:
        r = SESSION.get(url_hist, params=common, timeout=25); r.raise_for_status()
        data = r.json()
        if isinstance(data, list) and data:
            return data
    except Exception:
        pass
    # 2) fallback: odds (upcoming only)
    url_cur = "https://api.the-odds-api.com/v4/sports/americanfootball_nfl/odds"
    r = SESSION.get(url_cur, params=common, timeout=25); r.raise_for_status()
    data = r.json()
    return data if isinstance(data, list) else []

def extract_rows(ev: dict, year: int, week: int) -> list[dict]:
    """Return long-format rows for ALL bookmakers (so we 'take into account' all prices)."""
    home = to_key(ev.get("home_team",""))
    away = to_key(ev.get("away_team",""))
    commence = ev.get("commence_time")
    # Guard week by commence_time as an extra check
    rows = []
    if not home or not away:
        return rows
    books = ev.get("bookmakers", []) or []
    for bk in books:
        book_key = bk.get("key") or bk.get("title")
        # History may carry multiple snapshots; take the latest market (last by timestamp) if present
        markets = bk.get("markets", []) or []
        # Prefer latest h2h market
        h2h_markets = [m for m in markets if m.get("key")=="h2h"]
        if not h2h_markets:
            continue
        h2h = sorted(h2h_markets, key=lambda m: m.get("last_update",""))[-1]
        outcomes = h2h.get("outcomes", []) or []
        if len(outcomes) != 2:
            continue
        ml_map = { to_key(o.get("name","")): o.get("price") for o in outcomes }
        home_ml = ml_map.get(home); away_ml = ml_map.get(away)
        # Fallback if naming didn’t align
        if home_ml is None or away_ml is None:
            prices = [o.get("price") for o in outcomes if "price" in o]
            if len(prices)==2:
                away_ml, home_ml = prices[0], prices[1]
        rows += [
            {"year": year, "week": week, "team": home, "opponent": away, "home_away": "H",
             "ml": home_ml, "book": book_key, "commence_time": commence},
            {"year": year, "week": week, "team": away, "opponent": home, "home_away": "A",
             "ml": away_ml, "book": book_key, "commence_time": commence},
        ]
    return rows

# --------- Build all weeks ----------
all_rows = []
for w in WEEKS:
    events = fetch_week_events(YEAR, w)
    for ev in events:
        all_rows.extend(extract_rows(ev, YEAR, w))
    time.sleep(0.35)  # be polite to the API

odds_long = pd.DataFrame(all_rows)

# Deduplicate: keep the latest row per (week, team, opponent, book, commence_time)
if not odds_long.empty:
    odds_long = (odds_long
                 .drop_duplicates(subset=["week","team","opponent","book","commence_time"], keep="last")
                 .reset_index(drop=True))

# Optional: compute implied win prob here (per book); keep as separate column
def ml_to_prob(ml):
    try:
        ml = float(ml)
    except (TypeError, ValueError):
        return np.nan
    if ml < 0:  # favorite
        return (-ml) / ((-ml) + 100.0)
    return 100.0 / (ml + 100.0)

odds_long["win_prob_book"] = odds_long["ml"].map(ml_to_prob)

# Also create a book-aggregated view (DraftKings preferred; else mean-of-books)
def aggregate_books(df):
    # Prefer DraftKings; else average across books that exist for that team-week
    pref = df[df["book"].str.lower()=="draftkings"]
    if not pref.empty:
        return pref.iloc[0]
    return df.sort_values("win_prob_book", ascending=False).iloc[0]  # or use mean()

agg = (odds_long
       .groupby(["week","team","opponent","home_away"], group_keys=False)
       .apply(aggregate_books)
       .reset_index(drop=True))

# Save both: full-book detail and preferred-agg (useful for optimizer)
out_all = DATA_DIR / f"odds_long_allbooks_{YEAR}.csv"
out_pref = DATA_DIR / f"odds_long_{YEAR}.csv"
odds_long.to_csv(out_all, index=False)
agg.to_csv(out_pref, index=False)

print(f"Saved ALL books -> {out_all}  ({len(odds_long)} rows)")
print(f"Saved preferred odds -> {out_pref}  ({len(agg)} team-week rows)")
display(agg.head(12))

Saved ALL books -> ../data/odds_long_allbooks_2025.csv  (394 rows)
Saved preferred odds -> ../data/odds_long_2025.csv  (58 team-week rows)


/var/folders/6l/18s2w2wn2bd93c3r28h5qs_h0000gn/T/ipykernel_76794/1467001382.py:174: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(aggregate_books)


,year,week,team,opponent,home_away,ml,book,commence_time,win_prob_book
0,2025,4,Atlanta,Washington,H,-135,draftkings,2025-09-28T17:01:00Z,0.574468
1,2025,4,Baltimore,Kansas City,A,-148,draftkings,2025-09-28T20:25:00Z,0.596774
2,2025,4,Buffalo,New Orleans,H,-1450,draftkings,2025-09-28T17:01:00Z,0.935484
3,2025,4,Carolina,New England,A,205,draftkings,2025-09-28T17:01:00Z,0.327869
4,2025,4,Chicago,Las Vegas,A,100,draftkings,2025-09-28T20:26:00Z,0.500000
5,2025,4,Cincinnati,Denver,A,370,draftkings,2025-09-30T00:15:00Z,0.212766
6,2025,4,Cleveland,Detroit,A,400,draftkings,2025-09-28T17:00:00Z,0.200000
7,2025,4,Dallas,Green Bay,H,285,draftkings,2025-09-29T00:21:00Z,0.259740
8,2025,4,Denver,Cincinnati,H,-485,draftkings,2025-09-30T00:15:00Z,0.829060
9,2025,4,Detroit,Cleveland,H,-535,draftkings,2025-09-28T17:00:00Z,0.842520


### nans

In [6]:
# total NaNs in the whole DF
odds_long.isna().sum().sum()

# NaNs per column
odds_long.isna().sum()

# percentage of NaNs per column
odds_long.isna().mean() * 100

year             0.0
week             0.0
team             0.0
opponent         0.0
home_away        0.0
ml               0.0
book             0.0
commence_time    0.0
win_prob_book    0.0
dtype: float64

### Rank

###### CBS

In [7]:
# === Robust CBS Power Rankings -> cbs_rank_{YEAR}.csv ===
import re, sys
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pathlib import Path

YEAR = 2025
DATA = Path("../data"); DATA.mkdir(parents=True, exist_ok=True)
CBS_URL  = "https://www.cbssports.com/nfl/news/nfl-week-4-power-rankings-buccaneers-on-the-move/"
CBS_HTML = DATA / "cbs_power.html"   # optional local copy
OUT_PATH = DATA / f"cbs_rank_{YEAR}.csv"

print(f"[CBS] DATA dir: {DATA.resolve()}")
print(f"[CBS] Will write: {OUT_PATH.resolve()}")

TEAM_KEY = {
    # NFC (full + city + nickname)
    "Arizona Cardinals":"Arizona","Arizona":"Arizona","Cardinals":"Arizona","Cards":"Arizona",
    "Atlanta Falcons":"Atlanta","Atlanta":"Atlanta","Falcons":"Atlanta",
    "Carolina Panthers":"Carolina","Carolina":"Carolina","Panthers":"Carolina",
    "Chicago Bears":"Chicago","Chicago":"Chicago","Bears":"Chicago",
    "Dallas Cowboys":"Dallas","Dallas":"Dallas","Cowboys":"Dallas",
    "Detroit Lions":"Detroit","Detroit":"Detroit","Lions":"Detroit",
    "Green Bay Packers":"Green Bay","Green Bay":"Green Bay","Packers":"Green Bay",
    "Los Angeles Rams":"LA Rams","L.A. Rams":"LA Rams","LA Rams":"LA Rams","Rams":"LA Rams",
    "Minnesota Vikings":"Minnesota","Minnesota":"Minnesota","Vikings":"Minnesota",
    "New Orleans Saints":"New Orleans","New Orleans":"New Orleans","Saints":"New Orleans",
    "New York Giants":"NY Giants","NY Giants":"NY Giants","Giants":"NY Giants",
    "Philadelphia Eagles":"Philadelphia","Philadelphia":"Philadelphia","Eagles":"Philadelphia",
    "San Francisco 49ers":"San Francisco","San Francisco":"San Francisco","49ers":"San Francisco","Niners":"San Francisco",
    "Seattle Seahawks":"Seattle","Seattle":"Seattle","Seahawks":"Seattle",
    "Tampa Bay Buccaneers":"Tampa Bay","Tampa Bay":"Tampa Bay","Buccaneers":"Tampa Bay","Bucs":"Tampa Bay",
    "Washington Commanders":"Washington","Washington":"Washington","Commanders":"Washington",
    # AFC (full + city + nickname)
    "Baltimore Ravens":"Baltimore","Baltimore":"Baltimore","Ravens":"Baltimore",
    "Buffalo Bills":"Buffalo","Buffalo":"Buffalo","Bills":"Buffalo",
    "Cincinnati Bengals":"Cincinnati","Cincinnati":"Cincinnati","Bengals":"Cincinnati",
    "Cleveland Browns":"Cleveland","Cleveland":"Cleveland","Browns":"Cleveland",
    "Denver Broncos":"Denver","Denver":"Denver","Broncos":"Denver",
    "Houston Texans":"Houston","Houston":"Houston","Texans":"Houston",
    "Indianapolis Colts":"Indianapolis","Indianapolis":"Indianapolis","Colts":"Indianapolis",
    "Jacksonville Jaguars":"Jacksonville","Jacksonville":"Jacksonville","Jaguars":"Jacksonville","Jags":"Jacksonville",
    "Kansas City Chiefs":"Kansas City","Kansas City":"Kansas City","Chiefs":"Kansas City",
    "Las Vegas Raiders":"Las Vegas","Las Vegas":"Las Vegas","Raiders":"Las Vegas",
    "Los Angeles Chargers":"LA Chargers","L.A. Chargers":"LA Chargers","LA Chargers":"LA Chargers","Chargers":"LA Chargers","Bolts":"LA Chargers",
    "Miami Dolphins":"Miami","Miami":"Miami","Dolphins":"Miami","Fins":"Miami",
    "New England Patriots":"New England","New England":"New England","Patriots":"New England","Pats":"New England",
    "New York Jets":"NY Jets","NY Jets":"NY Jets","Jets":"NY Jets",
    "Pittsburgh Steelers":"Pittsburgh","Pittsburgh":"Pittsburgh","Steelers":"Pittsburgh",
    "Tennessee Titans":"Tennessee","Tennessee":"Tennessee","Titans":"Tennessee",
}
norm = lambda s: re.sub(r"\s+"," ",str(s)).strip()
to_key = lambda s: TEAM_KEY.get(norm(s), norm(s))

# ---------- 1) Load HTML with a session (less bot-blocking) ----------
if CBS_HTML.exists() and CBS_HTML.stat().st_size > 0:
    html = CBS_HTML.read_text(encoding="utf-8", errors="ignore")
    print(f"[CBS] Using local HTML: {CBS_HTML.resolve()}")
else:
    sess = requests.Session()
    sess.headers.update({
        "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/120 Safari/537.36"),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Referer": "https://www.google.com/",
    })
    r = sess.get(CBS_URL, timeout=25)
    r.raise_for_status()
    html = r.text
    # Optional: cache once to stabilize layout across runs
    try:
        CBS_HTML.write_text(html, encoding="utf-8")
        print(f"[CBS] Cached HTML -> {CBS_HTML.resolve()}")
    except Exception as e:
        print(f"[CBS] Warning: could not cache HTML: {e}", file=sys.stderr)

soup = BeautifulSoup(html, "lxml")

# ---------- 2) Try multiple parse strategies ----------
rows = []

# Strategy A: parse explicit tables with Rank + Team columns
for tbl in soup.select("table"):
    try:
        df_tbl = pd.read_html(str(tbl))[0]
    except ValueError:
        continue
    cols = [str(c).strip().lower() for c in df_tbl.columns]
    if any(c in ("rk","rank","#","ranking") for c in cols) and any("team" in c for c in cols):
        df_tbl.columns = [str(c).strip() for c in df_tbl.columns]
        rk_col = next(c for c in df_tbl.columns if c.lower() in ("rk","rank","#","ranking"))
        tm_col = next(c for c in df_tbl.columns if "team" in c.lower())
        for _, r_ in df_tbl.iterrows():
            rk = pd.to_numeric(r_.get(rk_col), errors="coerce")
            tm = norm(r_.get(tm_col))
            if pd.notna(rk) and tm:
                rows.append((int(rk), tm))
        if len(rows) >= 20:
            break

# Strategy B: scan short text nodes like "1 Team", "#1 Team", "1. Team"
def harvest_from_text_nodes(soup):
    out = []
    # Short text nodes first to avoid paragraph noise
    for node in soup.find_all(string=True):
        txt = norm(node)
        if not txt or len(txt) > 100:
            continue
        m = re.match(r"^#?\s*(\d{1,2})[.)]?\s+([A-Za-z .'-]+)$", txt)
        if m:
            out.append((int(m.group(1)), m.group(2)))
    return out

if len(rows) < 20:
    rows += harvest_from_text_nodes(soup)

# Strategy C: scan headings/strong/b for "1 Team" patterns
if len(rows) < 20:
    for tag in soup.find_all(re.compile(r"h[1-6]|strong|b|p|span|div")):
        txt = norm(tag.get_text(" "))
        m = re.match(r"^#?\s*(\d{1,2})[.)]?\s+([A-Za-z .'-]+)$", txt)
        if m:
            rows.append((int(m.group(1)), m.group(2)))

# ---------- 3) Clean & normalize ----------
if not rows:
    print("[CBS] ERROR: Parser found 0 rows. Save the page to ../data/cbs_power.html and rerun.", file=sys.stderr)
    # Still write an empty CSV so the pipeline doesn't break
    empty = pd.DataFrame(columns=["team_key","cbs_rank","team_raw"])
    empty.to_csv(OUT_PATH, index=False)
    print(f"[CBS] Wrote EMPTY CSV -> {OUT_PATH.resolve()}  (0 teams)")
else:
    cbs = (pd.DataFrame(rows, columns=["cbs_rank","team_raw"])
             .assign(cbs_rank=lambda d: pd.to_numeric(d["cbs_rank"], errors="coerce"))
             .dropna(subset=["cbs_rank"]))
    # strip junk and map keys
    cbs["team_raw"] = cbs["team_raw"].str.replace(r"\b\d+-\d+(?:-\d+)?\b", "", regex=True).str.strip()
    cbs["team_key"] = cbs["team_raw"].map(to_key)

    # keep one row per team (best rank) and filter to known NFL teams
    cbs = (cbs.sort_values("cbs_rank")
              .drop_duplicates("team_key", keep="first"))
    cbs = cbs[cbs["team_key"].isin(TEAM_KEY.values())]
    cbs["cbs_rank"] = cbs["cbs_rank"].astype(int)
    cbs = cbs[["team_key","cbs_rank","team_raw"]].sort_values("cbs_rank")

    # Sanity log
    n = len(cbs)
    if n < 28:
        print(f"[CBS] WARNING: Only {n} mapped teams parsed. Layout may have changed.", file=sys.stderr)
    elif n > 32:
        print(f"[CBS] WARNING: Parsed {n} teams (expected ~32). Duplicate/noise likely.", file=sys.stderr)

    # ---------- 4) Save (guaranteed) ----------
    cbs.to_csv(OUT_PATH, index=False)
    print(f"[CBS] Saved -> {OUT_PATH.resolve()}  ({n} teams)")
    display(cbs.head(32))

[CBS] DATA dir: /Users/santiagovillasenor/Library/CloudStorage/Dropbox-HeuristicsFinansoft/Jaime Villasenor/Personal/Santi/ITAM/9_Semestre/survivor/data
[CBS] Will write: /Users/santiagovillasenor/Library/CloudStorage/Dropbox-HeuristicsFinansoft/Jaime Villasenor/Personal/Santi/ITAM/9_Semestre/survivor/data/cbs_rank_2025.csv
[CBS] Cached HTML -> /Users/santiagovillasenor/Library/CloudStorage/Dropbox-HeuristicsFinansoft/Jaime Villasenor/Personal/Santi/ITAM/9_Semestre/survivor/data/cbs_power.html
[CBS] Saved -> /Users/santiagovillasenor/Library/CloudStorage/Dropbox-HeuristicsFinansoft/Jaime Villasenor/Personal/Santi/ITAM/9_Semestre/survivor/data/cbs_rank_2025.csv  (32 teams)


/var/folders/6l/18s2w2wn2bd93c3r28h5qs_h0000gn/T/ipykernel_76794/1305343738.py:87: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_tbl = pd.read_html(str(tbl))[0]


,team_key,cbs_rank,team_raw
0,Philadelphia,1,Eagles
1,Buffalo,2,Bills
2,Tampa Bay,3,Buccaneers
3,LA Chargers,4,Chargers
4,Green Bay,5,Packers
5,Detroit,6,Lions
6,Washington,7,Commanders
7,San Francisco,8,49ers
8,Indianapolis,9,Colts
9,LA Rams,10,Rams


###### FOX

In [8]:
# === FOX Sports Power Rankings -> fox_rank_{YEAR}.csv ===
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pathlib import Path

YEAR = 2025
DATA = Path("../data"); DATA.mkdir(parents=True, exist_ok=True)

FOX_URL  = "https://www.foxsports.com/stories/nfl/2025-nfl-power-rankings-week-4-how-many-teams-actually-great"
FOX_HTML = DATA / "fox_power.html"            # optional: save page once and it will use this
OUT_PATH = DATA / f"fox_rank_{YEAR}.csv"

TEAM_KEY = {
    # NFC
    "Arizona Cardinals":"Arizona","Arizona":"Arizona",
    "Atlanta Falcons":"Atlanta","Atlanta":"Atlanta",
    "Carolina Panthers":"Carolina","Carolina":"Carolina",
    "Chicago Bears":"Chicago","Chicago":"Chicago",
    "Dallas Cowboys":"Dallas","Dallas":"Dallas",
    "Detroit Lions":"Detroit","Detroit":"Detroit",
    "Green Bay Packers":"Green Bay","Green Bay":"Green Bay",
    "Los Angeles Rams":"LA Rams","L.A. Rams":"LA Rams","LA Rams":"LA Rams","Rams":"LA Rams",
    "Minnesota Vikings":"Minnesota","Minnesota":"Minnesota",
    "New Orleans Saints":"New Orleans","New Orleans":"New Orleans",
    "New York Giants":"NY Giants","NY Giants":"NY Giants","Giants":"NY Giants",
    "Philadelphia Eagles":"Philadelphia","Philadelphia":"Philadelphia","Eagles":"Philadelphia",
    "San Francisco 49ers":"San Francisco","San Francisco":"San Francisco","49ers":"San Francisco",
    "Seattle Seahawks":"Seattle","Seattle":"Seattle","Seahawks":"Seattle",
    "Tampa Bay Buccaneers":"Tampa Bay","Tampa Bay":"Tampa Bay","Buccaneers":"Tampa Bay","Bucs":"Tampa Bay",
    "Washington Commanders":"Washington","Washington":"Washington","Commanders":"Washington",
    # AFC
    "Baltimore Ravens":"Baltimore","Baltimore":"Baltimore","Ravens":"Baltimore",
    "Buffalo Bills":"Buffalo","Buffalo":"Buffalo","Bills":"Buffalo",
    "Cincinnati Bengals":"Cincinnati","Cincinnati":"Cincinnati","Bengals":"Cincinnati",
    "Cleveland Browns":"Cleveland","Cleveland":"Cleveland","Browns":"Cleveland",
    "Denver Broncos":"Denver","Denver":"Denver","Broncos":"Denver",
    "Houston Texans":"Houston","Houston":"Houston","Texans":"Houston",
    "Indianapolis Colts":"Indianapolis","Indianapolis":"Indianapolis","Colts":"Indianapolis",
    "Jacksonville Jaguars":"Jacksonville","Jacksonville":"Jacksonville","Jaguars":"Jacksonville","Jags":"Jacksonville",
    "Kansas City Chiefs":"Kansas City","Kansas City":"Kansas City","Chiefs":"Kansas City",
    "Las Vegas Raiders":"Las Vegas","Las Vegas":"Las Vegas","Raiders":"Las Vegas",
    "Los Angeles Chargers":"LA Chargers","L.A. Chargers":"LA Chargers","LA Chargers":"LA Chargers","Chargers":"LA Chargers",
    "Miami Dolphins":"Miami","Miami":"Miami","Dolphins":"Miami",
    "New England Patriots":"New England","New England":"New England","Patriots":"New England",
    "New York Jets":"NY Jets","NY Jets":"NY Jets","Jets":"NY Jets",
    "Pittsburgh Steelers":"Pittsburgh","Pittsburgh":"Pittsburgh","Steelers":"Pittsburgh",
    "Tennessee Titans":"Tennessee","Tennessee":"Tennessee","Titans":"Tennessee",
}
norm = lambda s: re.sub(r"\s+"," ",str(s)).strip()
to_key = lambda s: TEAM_KEY.get(norm(s), norm(s))

# --- load HTML (prefer local saved page if present) ---
if FOX_HTML.exists():
    html = FOX_HTML.read_text(encoding="utf-8", errors="ignore")
else:
    r = requests.get(FOX_URL, timeout=25)
    r.raise_for_status()
    html = r.text

soup = BeautifulSoup(html, "lxml")

# --- parse ranks ---
rows = []

# Collect short text nodes that start with "#<rank>"
for node in soup.find_all(string=True):
    txt = norm(node)
    if not txt or len(txt) > 100:  # skip long blobs
        continue
    if re.match(r"^#\s*\d{1,2}\b", txt):
        m = re.match(r"^#\s*(\d{1,2})\s+([A-Za-z .'-]+)", txt)
        if m:
            rows.append((int(m.group(1)), m.group(2)))

# Fallback: headings like "1. Team" or "1 Team"
if len(rows) < 20:
    for tag in soup.find_all(re.compile("h[1-6]|strong|b|p|span|div")):
        txt = norm(tag.get_text(" "))
        m = re.match(r"^#?\s*(\d{1,2})[.)]?\s+([A-Za-z .'-]+)$", txt)
        if m:
            rows.append((int(m.group(1)), m.group(2)))

if not rows:
    raise RuntimeError("FOX parse produced 0 rows. If needed, save the page to ../data/fox_power.html and rerun.")

# --- clean & save ---
fox = pd.DataFrame(rows, columns=["fox_rank","team_raw"]).dropna()
fox["fox_rank"] = pd.to_numeric(fox["fox_rank"], errors="coerce")
fox = fox.dropna(subset=["fox_rank"])
fox["fox_rank"] = fox["fox_rank"].astype(int)

fox["team_key"] = fox["team_raw"].map(to_key)
fox = fox.sort_values("fox_rank").drop_duplicates("team_key", keep="first")

# keep only mapped NFL teams
fox = fox[fox["team_key"].isin(TEAM_KEY.values())]
fox = fox[["team_key","fox_rank","team_raw"]].sort_values("fox_rank")

fox.to_csv(OUT_PATH, index=False)
print(f"Saved FOX ranks -> {OUT_PATH}  ({len(fox)} teams)")
display(fox.head(12))

Saved FOX ranks -> ../data/fox_rank_2025.csv  (32 teams)


,team_key,fox_rank,team_raw
0,Philadelphia,1,Philadelphia Eagles
1,Buffalo,2,Buffalo Bills
2,LA Chargers,3,Los Angeles Chargers
3,Detroit,4,Detroit Lions
4,Kansas City,5,Kansas City Chiefs
5,Tampa Bay,6,Tampa Bay Buccaneers
6,Baltimore,7,Baltimore Ravens
7,Green Bay,8,Green Bay Packers
8,Washington,9,Washington Commanders
9,LA Rams,10,Los Angeles Rams


###### NFL 

In [9]:
# === NFL.com Power Rankings -> nfl_rank_{YEAR}.csv (standalone) ===
import re
import pandas as pd# === NFL.com Power Rankings -> nfl_rank_{YEAR}.csv ===
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pathlib import Path

YEAR = 2025
DATA = Path("../data"); DATA.mkdir(parents=True, exist_ok=True)

NFL_URL  = "https://www.nfl.com/news/nfl-power-rankings-week-4-2025-nfl-season"
NFL_HTML = DATA / "nfl_power.html"              # optional: save page once and it will use this
OUT_PATH = DATA / f"nfl_rank_{YEAR}.csv"

TEAM_KEY = {
    # NFC
    "Arizona Cardinals":"Arizona","Arizona":"Arizona",
    "Atlanta Falcons":"Atlanta","Atlanta":"Atlanta",
    "Carolina Panthers":"Carolina","Carolina":"Carolina",
    "Chicago Bears":"Chicago","Chicago":"Chicago",
    "Dallas Cowboys":"Dallas","Dallas":"Dallas",
    "Detroit Lions":"Detroit","Detroit":"Detroit",
    "Green Bay Packers":"Green Bay","Green Bay":"Green Bay",
    "Los Angeles Rams":"LA Rams","L.A. Rams":"LA Rams","LA Rams":"LA Rams","Rams":"LA Rams",
    "Minnesota Vikings":"Minnesota","Minnesota":"Minnesota",
    "New Orleans Saints":"New Orleans","New Orleans":"New Orleans",
    "New York Giants":"NY Giants","NY Giants":"NY Giants",
    "Philadelphia Eagles":"Philadelphia","Philadelphia":"Philadelphia",
    "San Francisco 49ers":"San Francisco","San Francisco":"San Francisco","49ers":"San Francisco",
    "Seattle Seahawks":"Seattle","Seattle":"Seattle",
    "Tampa Bay Buccaneers":"Tampa Bay","Tampa Bay":"Tampa Bay","Buccaneers":"Tampa Bay","Bucs":"Tampa Bay",
    "Washington Commanders":"Washington","Washington":"Washington",
    # AFC
    "Baltimore Ravens":"Baltimore","Baltimore":"Baltimore",
    "Buffalo Bills":"Buffalo","Buffalo":"Buffalo",
    "Cincinnati Bengals":"Cincinnati","Cincinnati":"Cincinnati",
    "Cleveland Browns":"Cleveland","Cleveland":"Cleveland",
    "Denver Broncos":"Denver","Denver":"Denver",
    "Houston Texans":"Houston","Houston":"Houston",
    "Indianapolis Colts":"Indianapolis","Indianapolis":"Indianapolis",
    "Jacksonville Jaguars":"Jacksonville","Jacksonville":"Jacksonville","Jaguars":"Jacksonville","Jags":"Jacksonville",
    "Kansas City Chiefs":"Kansas City","Kansas City":"Kansas City",
    "Las Vegas Raiders":"Las Vegas","Las Vegas":"Las Vegas",
    "Los Angeles Chargers":"LA Chargers","L.A. Chargers":"LA Chargers","LA Chargers":"LA Chargers","Chargers":"LA Chargers",
    "Miami Dolphins":"Miami","Miami":"Miami",
    "New England Patriots":"New England","New England":"New England",
    "New York Jets":"NY Jets","NY Jets":"NY Jets",
    "Pittsburgh Steelers":"Pittsburgh","Pittsburgh":"Pittsburgh",
    "Tennessee Titans":"Tennessee","Tennessee":"Tennessee",
}
norm = lambda s: re.sub(r"\s+"," ",str(s)).strip()
to_key = lambda s: TEAM_KEY.get(norm(s), norm(s))

# --- load HTML (prefer local saved page if present) ---
if NFL_HTML.exists():
    html = NFL_HTML.read_text(encoding="utf-8", errors="ignore")
else:
    sess = requests.Session()
    sess.headers.update({
        "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/120 Safari/537.36"),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Referer": "https://www.google.com/",
    })
    r = sess.get(NFL_URL, timeout=25)
    r.raise_for_status()
    html = r.text

soup = BeautifulSoup(html, "lxml")

# --- parse ranks ---
# NFL.com card layout often has "Rank" then the number, then the Team name a few nodes later.
lines = [norm(x) for x in soup.get_text("\n").splitlines()]
lines = [ln for ln in lines if ln]

rows = []
i = 0
while i < len(lines):
    if lines[i].lower() == "rank":
        # find next pure rank number
        j = i + 1
        while j < len(lines) and not re.match(r"^\d{1,2}$", lines[j]):
            j += 1
        if j < len(lines):
            rk = int(lines[j])
            # next plausible team name (skip records like 3-0)
            k = j + 1
            team = None
            while k < len(lines):
                cand = lines[k]
                if re.match(r"^\d{1,2}$", cand):
                    k += 1; continue
                if re.search(r"\b\d+-\d+(?:-\d+)?\b", cand):
                    k += 1; continue
                team = cand
                break
            if team:
                team = re.sub(r"\s*\(.*?\)\s*$", "", team)
                rows.append((rk, team))
                i = k
                continue
    i += 1

# Fallback: also match lines like "1. Team" or "#1 Team"
if len(rows) < 20:
    for ln in lines:
        m = re.match(r"^#?\s*(\d{1,2})[.)]?\s+([A-Za-z .'-]+)$", ln)
        if m:
            rows.append((int(m.group(1)), m.group(2)))

if not rows:
    raise RuntimeError("NFL.com parser found 0 rows. If needed, save the page to ../data/nfl_power.html and rerun.")

# --- clean & save ---
df = pd.DataFrame(rows, columns=["nfl_rank","team_raw"]).dropna()
df["nfl_rank"] = pd.to_numeric(df["nfl_rank"], errors="coerce")
df = df.dropna(subset=["nfl_rank"])
df["nfl_rank"] = df["nfl_rank"].astype(int)

# Remove record fragments & normalize names
df["team_raw"] = df["team_raw"].str.replace(r"\b\d+-\d+(?:-\d+)?\b", "", regex=True).str.strip()
df["team_key"] = df["team_raw"].map(to_key)

# One row per team (lowest rank wins), only mapped NFL clubs
df = df.sort_values("nfl_rank").drop_duplicates("team_key", keep="first")
df = df[df["team_key"].isin(TEAM_KEY.values())]
df = df[["team_key","nfl_rank","team_raw"]].sort_values("nfl_rank")

df.to_csv(OUT_PATH, index=False)
print(f"Saved NFL.com ranks -> {OUT_PATH}  ({len(df)} teams)")
display(df.head(12))
import requests
from bs4 import BeautifulSoup
from pathlib import Path

YEAR = 2025
DATA = Path("../data"); DATA.mkdir(parents=True, exist_ok=True)

NFL_URL  = "https://www.nfl.com/news/nfl-power-rankings-week-4-2025-nfl-season"
NFL_HTML = DATA / "nfl_power.html"   # optional: save page once to avoid layout changes/anti-bot
OUT_PATH = DATA / f"nfl_rank_{YEAR}.csv"

# Map names to your schedule's city-style keys (same mapping you’ve used)
TEAM_KEY = {
    "Arizona Cardinals":"Arizona","Arizona":"Arizona",
    "Atlanta Falcons":"Atlanta","Atlanta":"Atlanta",
    "Carolina Panthers":"Carolina","Carolina":"Carolina",
    "Chicago Bears":"Chicago","Chicago":"Chicago",
    "Dallas Cowboys":"Dallas","Dallas":"Dallas",
    "Detroit Lions":"Detroit","Detroit":"Detroit",
    "Green Bay Packers":"Green Bay","Green Bay":"Green Bay",
    "Los Angeles Rams":"LA Rams","L.A. Rams":"LA Rams","LA Rams":"LA Rams","Rams":"LA Rams",
    "Minnesota Vikings":"Minnesota","Minnesota":"Minnesota",
    "New Orleans Saints":"New Orleans","New Orleans":"New Orleans",
    "New York Giants":"NY Giants","NY Giants":"NY Giants",
    "Philadelphia Eagles":"Philadelphia","Philadelphia":"Philadelphia",
    "San Francisco 49ers":"San Francisco","San Francisco":"San Francisco","49ers":"San Francisco",
    "Seattle Seahawks":"Seattle","Seattle":"Seattle",
    "Tampa Bay Buccaneers":"Tampa Bay","Tampa Bay":"Tampa Bay","Buccaneers":"Tampa Bay","Bucs":"Tampa Bay",
    "Washington Commanders":"Washington","Washington":"Washington",
    "Baltimore Ravens":"Baltimore","Baltimore":"Baltimore",
    "Buffalo Bills":"Buffalo","Buffalo":"Buffalo",
    "Cincinnati Bengals":"Cincinnati","Cincinnati":"Cincinnati",
    "Cleveland Browns":"Cleveland","Cleveland":"Cleveland",
    "Denver Broncos":"Denver","Denver":"Denver",
    "Houston Texans":"Houston","Houston":"Houston",
    "Indianapolis Colts":"Indianapolis","Indianapolis":"Indianapolis",
    "Jacksonville Jaguars":"Jacksonville","Jacksonville":"Jacksonville","Jaguars":"Jacksonville","Jags":"Jacksonville",
    "Kansas City Chiefs":"Kansas City","Kansas City":"Kansas City",
    "Las Vegas Raiders":"Las Vegas","Las Vegas":"Las Vegas",
    "Los Angeles Chargers":"LA Chargers","L.A. Chargers":"LA Chargers","LA Chargers":"LA Chargers","Chargers":"LA Chargers",
    "Miami Dolphins":"Miami","Miami":"Miami",
    "New England Patriots":"New England","New England":"New England",
    "New York Jets":"NY Jets","NY Jets":"NY Jets",
    "Pittsburgh Steelers":"Pittsburgh","Pittsburgh":"Pittsburgh",
    "Tennessee Titans":"Tennessee","Tennessee":"Tennessee",
}
norm = lambda s: re.sub(r"\s+"," ",str(s)).strip()
to_key = lambda s: TEAM_KEY.get(norm(s), norm(s))

# 1) Load HTML (prefer local saved copy if present)
if NFL_HTML.exists():
    html = NFL_HTML.read_text(encoding="utf-8", errors="ignore")
else:
    sess = requests.Session()
    sess.headers.update({
        "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/120 Safari/537.36"),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Referer": "https://www.google.com/",
    })
    r = sess.get(NFL_URL, timeout=25)
    r.raise_for_status()
    html = r.text

soup = BeautifulSoup(html, "lxml")

# 2) Parse: NFL.com cards usually show blocks like:
#    "Rank" / "1" / "<Team Name>" / "<record>"
# We'll scan text lines and use a sliding window: when we see "Rank", next non-empty is number, then team.
lines = [norm(x) for x in soup.get_text("\n").splitlines()]
lines = [ln for ln in lines if ln]  # drop empties

rows = []
i = 0
while i < len(lines):
    if lines[i].lower() == "rank":
        # find next numeric line
        j = i + 1
        while j < len(lines) and not re.match(r"^\d{1,2}$", lines[j]):  # pure rank number
            j += 1
        if j < len(lines):
            rk = int(lines[j])
            # next non-empty, non-number line should be team
            k = j + 1
            # skip record lines like "3-0" or "2-1"
            team = None
            while k < len(lines):
                cand = lines[k]
                if re.match(r"^\d{1,2}$", cand):  # another number; unlikely but skip
                    k += 1; continue
                if re.search(r"\b\d+-\d+(?:-\d+)?\b", cand):  # records like 3-0 or 10-6-1
                    k += 1; continue
                team = cand
                break
            if team:
                # clean possible tail like "Rank" or odd labels
                team = re.sub(r"\s*\(.*?\)\s*$", "", team)
                rows.append((rk, team))
                # advance a bit to avoid duplicates
                i = k
                continue
    i += 1

# Fallback: also consider headings like "#1 Team" or "1. Team"
if len(rows) < 20:
    for ln in lines:
        m = re.match(r"^#?\s*(\d{1,2})[.)]?\s+([A-Za-z .'-]+)$", ln)
        if m:
            rows.append((int(m.group(1)), m.group(2)))

if not rows:
    raise RuntimeError("NFL.com parser found 0 rows. If needed, save the page to ../data/nfl_power.html and rerun.")

# 3) Clean & normalize
df = pd.DataFrame(rows, columns=["nfl_rank","team_raw"]).dropna()
df["nfl_rank"] = pd.to_numeric(df["nfl_rank"], errors="coerce")
df = df.dropna(subset=["nfl_rank"])
df["nfl_rank"] = df["nfl_rank"].astype(int)

# Normalize team names
# Strip trailing records that slipped in, and map to keys
df["team_raw"] = df["team_raw"].str.replace(r"\b\d+-\d+(?:-\d+)?\b", "", regex=True).str.strip()
df["team_key"] = df["team_raw"].map(to_key)

# Keep one row per team -> best (lowest) rank if duplicates
df = df.sort_values("nfl_rank").drop_duplicates("team_key", keep="first")

# Filter to known teams and keep final columns
df = df[df["team_key"].isin(TEAM_KEY.values())]
df = df[["team_key","nfl_rank","team_raw"]].sort_values("nfl_rank")

# 4) Save
df.to_csv(OUT_PATH, index=False)
print(f"Saved NFL.com ranks -> {OUT_PATH}  ({len(df)} teams)")
display(df.head(12))

Saved NFL.com ranks -> ../data/nfl_rank_2025.csv  (32 teams)


,team_key,nfl_rank,team_raw
0,Philadelphia,1,Philadelphia Eagles
1,Buffalo,2,Buffalo Bills
2,Detroit,3,Detroit Lions
3,LA Chargers,4,Los Angeles Chargers
4,Tampa Bay,5,Tampa Bay Buccaneers
5,Green Bay,6,Green Bay Packers
6,Baltimore,7,Baltimore Ravens
7,Washington,8,Washington Commanders
8,Indianapolis,9,Indianapolis Colts
9,LA Rams,10,Los Angeles Rams


Saved NFL.com ranks -> ../data/nfl_rank_2025.csv  (32 teams)


,team_key,nfl_rank,team_raw
0,Philadelphia,1,Philadelphia Eagles
1,Buffalo,2,Buffalo Bills
2,Detroit,3,Detroit Lions
3,LA Chargers,4,Los Angeles Chargers
4,Tampa Bay,5,Tampa Bay Buccaneers
5,Green Bay,6,Green Bay Packers
6,Baltimore,7,Baltimore Ravens
7,Washington,8,Washington Commanders
8,Indianapolis,9,Indianapolis Colts
9,LA Rams,10,Los Angeles Rams


Bleacher report

In [10]:
# === Bleacher Report Power Rankings -> br_rank_{YEAR}.csv (digit-safe) ===
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pathlib import Path

YEAR = 2025
DATA = Path("../data"); DATA.mkdir(parents=True, exist_ok=True)

BR_URL  = "https://bleacherreport.com/articles/25251498-br-experts-week-4-nfl-power-rankings" # Has to be changed each week
BR_HTML = DATA / "br_power.html"          # optional: save once locally
OUT_PATH = DATA / f"br_rank_{YEAR}.csv"

TEAM_KEY = {
    # NFC
    "Arizona Cardinals":"Arizona","Arizona":"Arizona",
    "Atlanta Falcons":"Atlanta","Atlanta":"Atlanta",
    "Carolina Panthers":"Carolina","Carolina":"Carolina",
    "Chicago Bears":"Chicago","Chicago":"Chicago",
    "Dallas Cowboys":"Dallas","Dallas":"Dallas",
    "Detroit Lions":"Detroit","Detroit":"Detroit",
    "Green Bay Packers":"Green Bay","Green Bay":"Green Bay",
    "Los Angeles Rams":"LA Rams","L.A. Rams":"LA Rams","LA Rams":"LA Rams","Rams":"LA Rams",
    "Minnesota Vikings":"Minnesota","Minnesota":"Minnesota",
    "New Orleans Saints":"New Orleans","New Orleans":"New Orleans",
    "New York Giants":"NY Giants","NY Giants":"NY Giants","Giants":"NY Giants",
    "Philadelphia Eagles":"Philadelphia","Philadelphia":"Philadelphia",
    "San Francisco 49ers":"San Francisco","San Francisco":"San Francisco","49ers":"San Francisco",
    "Seattle Seahawks":"Seattle","Seattle":"Seattle",
    "Tampa Bay Buccaneers":"Tampa Bay","Tampa Bay":"Tampa Bay","Buccaneers":"Tampa Bay","Bucs":"Tampa Bay",
    "Washington Commanders":"Washington","Washington":"Washington",
    # AFC
    "Baltimore Ravens":"Baltimore","Baltimore":"Baltimore",
    "Buffalo Bills":"Buffalo","Buffalo":"Buffalo",
    "Cincinnati Bengals":"Cincinnati","Cincinnati":"Cincinnati",
    "Cleveland Browns":"Cleveland","Cleveland":"Cleveland",
    "Denver Broncos":"Denver","Denver":"Denver",
    "Houston Texans":"Houston","Houston":"Houston",
    "Indianapolis Colts":"Indianapolis","Indianapolis":"Indianapolis",
    "Jacksonville Jaguars":"Jacksonville","Jacksonville":"Jacksonville","Jaguars":"Jacksonville","Jags":"Jacksonville",
    "Kansas City Chiefs":"Kansas City","Kansas City":"Kansas City",
    "Las Vegas Raiders":"Las Vegas","Las Vegas":"Las Vegas","Raiders":"Las Vegas",
    "Los Angeles Chargers":"LA Chargers","L.A. Chargers":"LA Chargers","LA Chargers":"LA Chargers","Chargers":"LA Chargers",
    "Miami Dolphins":"Miami","Miami":"Miami",
    "New England Patriots":"New England","New England":"New England",
    "New York Jets":"NY Jets","NY Jets":"NY Jets","Jets":"NY Jets",
    "Pittsburgh Steelers":"Pittsburgh","Pittsburgh":"Pittsburgh",
    "Tennessee Titans":"Tennessee","Tennessee":"Tennessee",
}
norm = lambda s: re.sub(r"\s+"," ", str(s or "")).strip()
to_key = lambda s: TEAM_KEY.get(norm(s), norm(s))

# --- load HTML (prefer local saved copy if present) ---
if BR_HTML.exists():
    html = BR_HTML.read_text(encoding="utf-8", errors="ignore")
else:
    sess = requests.Session()
    sess.headers.update({
        "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                       "AppleWebKit/537.36 (KHTML, like Gecko) "
                       "Chrome/120 Safari/537.36"),
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
        "Referer": "https://www.google.com/",
    })
    r = sess.get(BR_URL, timeout=25)
    r.raise_for_status()
    html = r.text

soup = BeautifulSoup(html, "lxml")

# --- parse ranks (digit-safe + unicode punctuation) ---
rows = []

# Allow digits in team names, plus curly apostrophes and en dashes
TEAM_CHARS = r"[A-Za-z0-9 .'\-\u2019\u2013]+"

lines = [norm(x) for x in soup.get_text("\n").splitlines()]
lines = [ln for ln in lines if ln]

patts = [
    # "No. 1 Team" / "#1 Team" / "1. Team" / "1) Team" / "1 – Team" / "1 - Team" / "1: Team"
    rf"^\s*(?:No\.\s*|#\s*)?(\d{{1,2}})[.)]?\s+({TEAM_CHARS}?)(?:\s*\(.*?\))?\s*$",
    rf"^\s*(\d{{1,2}})\s*[-:\u2013]\s*({TEAM_CHARS}?)(?:\s*\(.*?\))?\s*$",
]

for ln in lines:
    for p in patts:
        m = re.match(p, ln)
        if m:
            rank = int(m.group(1))
            # strip any trailing annotations like "(Last week: 2)"
            name = re.sub(r"\(.*?\)\s*$", "", m.group(2)).strip()
            rows.append((rank, name))
            break

# Fallback: scan headline-ish tags again
if len(rows) < 20:
    for tag in soup.find_all(re.compile("h[1-6]|strong|b|p|span|div|li")):
        txt = norm(tag.get_text(" "))
        for p in patts:
            m = re.match(p, txt)
            if m:
                rank = int(m.group(1))
                name = re.sub(r"\(.*?\)\s*$", "", m.group(2)).strip()
                rows.append((rank, name))
                break

if not rows:
    raise RuntimeError("Bleacher Report parser found 0 rows. Consider saving the page as ../data/br_power.html and rerun.")

# --- clean & normalize ---
df = (pd.DataFrame(rows, columns=["br_rank","team_raw"])
        .assign(br_rank=lambda d: pd.to_numeric(d["br_rank"], errors="coerce"))
        .dropna(subset=["br_rank"]))

# remove embedded records like "3-0"
df["team_raw"] = (df["team_raw"]
                  .str.replace(r"\b\d+-\d+(?:-\d+)?\b", "", regex=True)
                  .str.replace(r"\s+", " ", regex=True)
                  .str.strip())

df["team_key"] = df["team_raw"].map(to_key)

# One row per team (lowest rank kept), keep only mapped NFL clubs
df = (df.sort_values("br_rank")
        .drop_duplicates("team_key", keep="first"))
df = df[df["team_key"].isin(TEAM_KEY.values())]
df = df[["team_key","br_rank","team_raw"]].sort_values("br_rank").reset_index(drop=True)

# --- diagnostics: who’s missing? ---
expected = set(TEAM_KEY.values())
have = set(df["team_key"])
missing = sorted(expected - have)
print(f"Saved Bleacher Report ranks -> {OUT_PATH}  ({len(df)} teams)")
if missing:
    print("Missing mapped teams:", missing)

# save
df.to_csv(OUT_PATH, index=False)
display(df.head(12))

Saved Bleacher Report ranks -> ../data/br_rank_2025.csv  (32 teams)


,team_key,br_rank,team_raw
0,Philadelphia,1,Philadelphia Eagles
1,Buffalo,2,Buffalo Bills
2,LA Chargers,3,Los Angeles Chargers
3,Tampa Bay,4,Tampa Bay Buccaneers
4,Detroit,5,Detroit Lions
5,Green Bay,6,Green Bay Packers
6,LA Rams,7,Los Angeles Rams
7,Baltimore,8,Baltimore Ravens
8,Kansas City,9,Kansas City Chiefs
9,San Francisco,10,San Francisco 49ers


## Average Power rank

In [69]:
# === Consensus Power Rankings (CBS + Fox + NFL + BR) ===
import pandas as pd
from pathlib import Path
import re

YEAR = 2025
DATA = Path("../data")

# --- TEAM_KEY normalization (same as before, unified) ---
TEAM_KEY = {
    "Arizona Cardinals":"Arizona","Arizona":"Arizona",
    "Atlanta Falcons":"Atlanta","Atlanta":"Atlanta",
    "Carolina Panthers":"Carolina","Carolina":"Carolina",
    "Chicago Bears":"Chicago","Chicago":"Chicago",
    "Dallas Cowboys":"Dallas","Dallas":"Dallas",
    "Detroit Lions":"Detroit","Detroit":"Detroit",
    "Green Bay Packers":"Green Bay","Green Bay":"Green Bay",
    "Los Angeles Rams":"LA Rams","L.A. Rams":"LA Rams","LA Rams":"LA Rams","Rams":"LA Rams",
    "Minnesota Vikings":"Minnesota","Minnesota":"Minnesota",
    "New Orleans Saints":"New Orleans","New Orleans":"New Orleans",
    "New York Giants":"NY Giants","NY Giants":"NY Giants","Giants":"NY Giants",
    "Philadelphia Eagles":"Philadelphia","Philadelphia":"Philadelphia",
    "San Francisco 49ers":"San Francisco","San Francisco":"San Francisco","49ers":"San Francisco",
    "Seattle Seahawks":"Seattle","Seattle":"Seattle",
    "Tampa Bay Buccaneers":"Tampa Bay","Tampa Bay":"Tampa Bay","Buccaneers":"Tampa Bay","Bucs":"Tampa Bay",
    "Washington Commanders":"Washington","Washington":"Washington",
    "Baltimore Ravens":"Baltimore","Baltimore":"Baltimore",
    "Buffalo Bills":"Buffalo","Buffalo":"Buffalo",
    "Cincinnati Bengals":"Cincinnati","Cincinnati":"Cincinnati",
    "Cleveland Browns":"Cleveland","Cleveland":"Cleveland",
    "Denver Broncos":"Denver","Denver":"Denver",
    "Houston Texans":"Houston","Houston":"Houston",
    "Indianapolis Colts":"Indianapolis","Indianapolis":"Indianapolis",
    "Jacksonville Jaguars":"Jacksonville","Jacksonville":"Jacksonville","Jaguars":"Jacksonville","Jags":"Jacksonville",
    "Kansas City Chiefs":"Kansas City","Kansas City":"Kansas City",
    "Las Vegas Raiders":"Las Vegas","Las Vegas":"Las Vegas","Raiders":"Las Vegas",
    "Los Angeles Chargers":"LA Chargers","L.A. Chargers":"LA Chargers","LA Chargers":"LA Chargers","Chargers":"LA Chargers",
    "Miami Dolphins":"Miami","Miami":"Miami",
    "New England Patriots":"New England","New England":"New England",
    "New York Jets":"NY Jets","NY Jets":"NY Jets","Jets":"NY Jets",
    "Pittsburgh Steelers":"Pittsburgh","Pittsburgh":"Pittsburgh",
    "Tennessee Titans":"Tennessee","Tennessee":"Tennessee",
}
norm = lambda s: re.sub(r"\s+"," ", str(s or "")).strip()
to_key = lambda s: TEAM_KEY.get(norm(s), norm(s))

# --- Load each source and normalize ---
def load_and_norm(path, colname):
    df = pd.read_csv(path)
    # make sure there's a "team_key" column
    if "team_key" not in df.columns:
        team_col = [c for c in df.columns if "team" in c.lower()][0]
        df["team_key"] = df[team_col].map(to_key)
    df = df[["team_key", colname]]
    return df

cbs = load_and_norm(DATA / f"cbs_rank_{YEAR}.csv", "cbs_rank")
fox = load_and_norm(DATA / f"fox_rank_{YEAR}.csv", "fox_rank")
nfl = load_and_norm(DATA / f"nfl_rank_{YEAR}.csv", "nfl_rank")
br  = load_and_norm(DATA / f"br_rank_{YEAR}.csv",  "br_rank")

# --- Merge all sources on normalized team_key ---
consensus = cbs.merge(fox, on="team_key", how="outer") \
               .merge(nfl, on="team_key", how="outer") \
               .merge(br,  on="team_key", how="outer")

# --- Compute consensus ---
consensus["consensus_rank"] = consensus[
    ["cbs_rank","fox_rank","nfl_rank","br_rank"]
].mean(axis=1, skipna=True)

consensus["consensus_score"] = 33 - consensus["consensus_rank"]

# sort best -> worst
consensus = consensus.sort_values("consensus_rank").reset_index(drop=True)

# save
out = DATA / f"consensus_rank_{YEAR}.csv"
consensus.to_csv(out, index=False)
print(f"Saved consensus ranks -> {out} ({len(consensus)} teams)")
display(consensus.head(5))

Saved consensus ranks -> ../data/consensus_rank_2025.csv (32 teams)


,team_key,cbs_rank,fox_rank,nfl_rank,br_rank,consensus_rank,consensus_score
0,Philadelphia,1,1,1,1,1.0,32.0
1,Buffalo,2,2,2,2,2.0,31.0
2,LA Chargers,4,3,4,3,3.5,29.5
3,Tampa Bay,3,6,5,4,4.5,28.5
4,Detroit,6,4,3,5,4.5,28.5


# Offense and Defense

> Offense

In [37]:
# === FOX Sports Offense Tables (collision-safe) -> offense_fox_{YEAR}.csv ===
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from pathlib import Path

YEAR = 2025
DATA = Path("../data"); DATA.mkdir(parents=True, exist_ok=True)
FOX_URL = "https://www.foxsports.com/articles/nfl/2025-nfl-offense-rankings-team-pass-and-rush-stats" # Update as needed
OUT_PATH = DATA / f"offense_fox_{YEAR}.csv"

# ---- Team mapping: FOX labels -> your city-style keys ----
TEAM_KEY = {
    "Arizona Cardinals":"Arizona","Arizona":"Arizona","Cardinals":"Arizona",
    "Atlanta Falcons":"Atlanta","Atlanta":"Atlanta","Falcons":"Atlanta",
    "Carolina Panthers":"Carolina","Carolina":"Carolina","Panthers":"Carolina",
    "Chicago Bears":"Chicago","Chicago":"Chicago","Bears":"Chicago",
    "Dallas Cowboys":"Dallas","Dallas":"Dallas","Cowboys":"Dallas",
    "Detroit Lions":"Detroit","Detroit":"Detroit","Lions":"Detroit",
    "Green Bay Packers":"Green Bay","Green Bay":"Green Bay","Packers":"Green Bay",
    "Los Angeles Rams":"LA Rams","L.A. Rams":"LA Rams","LA Rams":"LA Rams","Rams":"LA Rams",
    "Minnesota Vikings":"Minnesota","Minnesota":"Minnesota","Vikings":"Minnesota",
    "New Orleans Saints":"New Orleans","New Orleans":"New Orleans","Saints":"New Orleans",
    "New York Giants":"NY Giants","NY Giants":"NY Giants","Giants":"NY Giants",
    "Philadelphia Eagles":"Philadelphia","Philadelphia":"Philadelphia","Eagles":"Philadelphia",
    "San Francisco 49ers":"San Francisco","San Francisco":"San Francisco","49ers":"San Francisco",
    "Seattle Seahawks":"Seattle","Seattle":"Seattle","Seahawks":"Seattle",
    "Tampa Bay Buccaneers":"Tampa Bay","Tampa Bay":"Tampa Bay","Buccaneers":"Tampa Bay","Bucs":"Tampa Bay",
    "Washington Commanders":"Washington","Washington":"Washington","Commanders":"Washington",

    "Baltimore Ravens":"Baltimore","Baltimore":"Baltimore","Ravens":"Baltimore",
    "Buffalo Bills":"Buffalo","Buffalo":"Buffalo","Bills":"Buffalo",
    "Cincinnati Bengals":"Cincinnati","Cincinnati":"Cincinnati","Bengals":"Cincinnati",
    "Cleveland Browns":"Cleveland","Cleveland":"Cleveland","Browns":"Cleveland",
    "Denver Broncos":"Denver","Denver":"Denver","Broncos":"Denver",
    "Houston Texans":"Houston","Houston":"Houston","Texans":"Houston",
    "Indianapolis Colts":"Indianapolis","Indianapolis":"Indianapolis","Colts":"Indianapolis",
    "Jacksonville Jaguars":"Jacksonville","Jacksonville":"Jacksonville","Jaguars":"Jacksonville",
    "Kansas City Chiefs":"Kansas City","Kansas City":"Kansas City","Chiefs":"Kansas City",
    "Las Vegas Raiders":"Las Vegas","Las Vegas":"Las Vegas","Raiders":"Las Vegas",
    "Los Angeles Chargers":"LA Chargers","L.A. Chargers":"LA Chargers","LA Chargers":"LA Chargers","Chargers":"LA Chargers",
    "Miami Dolphins":"Miami","Miami":"Miami","Dolphins":"Miami",
    "New England Patriots":"New England","New England":"New England","Patriots":"New England",
    "New York Jets":"NY Jets","NY Jets":"NY Jets","Jets":"NY Jets",
    "Pittsburgh Steelers":"Pittsburgh","Pittsburgh":"Pittsburgh","Steelers":"Pittsburgh",
    "Tennessee Titans":"Tennessee","Tennessee":"Tennessee","Titans":"Tennessee",
}
norm = lambda s: re.sub(r"\s+"," ", str(s or "")).strip()
to_key = lambda s: TEAM_KEY.get(norm(s), norm(s))

# ---- fetch ----
sess = requests.Session()
sess.headers.update({
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/120.0.0.0 Safari/537.36")
})
r = sess.get(FOX_URL, timeout=25); r.raise_for_status()
soup = BeautifulSoup(r.text, "lxml")

def table_after_heading(heading_regex: str):
    h = soup.find(lambda tag: tag.name in {"h1","h2","h3","h4"} and re.search(heading_regex, tag.get_text(" "), re.I))
    return None if not h else h.find_next("table")

def parse_named_table(heading_regex: str, rename_map: dict, value_cols: list):
    """
    Return ONLY team_key + the requested value_cols (coerced numeric).
    This avoids carrying rank/# columns that collide on merge.
    """
    tbl = table_after_heading(heading_regex)
    if tbl is None:
        return pd.DataFrame(columns=["team_key"] + value_cols)
    df = pd.read_html(str(tbl))[0]
    df.columns = [norm(c).lower() for c in df.columns]

    # Map headers to our target names (best effort, via regex keys)
    col_map = {}
    for patt, target in rename_map.items():
        hit = next((c for c in df.columns if re.search(patt, c, re.I)), None)
        if hit:
            col_map[hit] = target
    df = df.rename(columns=col_map)

    # Find team column if needed
    if "team" not in df.columns:
        cand = next((c for c in df.columns if "team" in c or "club" in c), None)
        if cand: df = df.rename(columns={cand:"team"})
    if "team" not in df.columns:
        return pd.DataFrame(columns=["team_key"] + value_cols)

    # Map team to key
    df["team_key"] = df["team"].map(to_key)
    df = df[df["team_key"].isin(set(TEAM_KEY.values()))]

    # Keep only requested value columns that we actually found
    keep_vals = [c for c in value_cols if c in df.columns]
    out = df[["team_key"] + keep_vals].copy()

    # Coerce numeric
    for vc in keep_vals:
        out[vc] = (
            out[vc].astype(str)
                  .str.replace(",","", regex=False)
                  .str.replace("%","", regex=False)
                  .str.replace("T","", regex=False)   # strip 'T' in long plays if present
        )
        out[vc] = pd.to_numeric(out[vc], errors="coerce")

    return out

# Sections (regex anchored on headings)
total_off = parse_named_table(
    r"Total Offense",
    rename_map={r"team":"team", r"offense.*yards.*avg": "yds_g", r"offense.*td": "off_td"},
    value_cols=["yds_g","off_td"]
)
pass_off = parse_named_table(
    r"Passing Offense",
    rename_map={r"team":"team", r"pass.*yards.*avg": "pass_yds_g", r"pass.*td": "pass_td"},
    value_cols=["pass_yds_g","pass_td"]
)
rush_off = parse_named_table(
    r"Rushing Offense",
    rename_map={r"team":"team", r"rush.*yards.*avg": "rush_yds_g", r"rush.*td": "rush_td"},
    value_cols=["rush_yds_g","rush_td"]
)
score_off = parse_named_table(
    r"Scoring Offense",
    rename_map={r"team":"team", r"points.*scored.*avg": "pts_g", r"total.*td": "total_td"},
    value_cols=["pts_g","total_td"]
)
rz_off = parse_named_table(
    r"Red Zone Offense",
    rename_map={r"team":"team", r"red.*zone.*td.*rate": "rz_td_pct"},
    value_cols=["rz_td_pct"]
)

# Combine safely: start with team list, then left-merge value-only frames
base = pd.DataFrame({"team_key": sorted(set(TEAM_KEY.values()))})
for d in [total_off, pass_off, rush_off, score_off, rz_off]:
    base = base.merge(d, on="team_key", how="left")

# Ranks (higher is better)
def rank_desc(s):  # 1 best
    return s.rank(method="min", ascending=False)

for c in ["yds_g","off_td","pass_yds_g","pass_td","rush_yds_g","rush_td","pts_g","total_td","rz_td_pct"]:
    if c in base.columns:
        base[f"{c}_rank"] = rank_desc(base[c])

# Quick consensus offense score (z-avg of selected components; adjust later if you want weights)
components = [c for c in ["yds_g","pass_yds_g","rush_yds_g","pts_g","rz_td_pct"] if c in base.columns]
for c in components:
    base[f"z_{c}"] = (base[c] - base[c].mean()) / base[c].std(ddof=0)
if components:
    base["offense_consensus_score"] = base[[f"z_{c}" for c in components]].mean(axis=1)
    base["offense_consensus_rank"]  = base["offense_consensus_score"].rank(method="min", ascending=False)

# Save
base = base.sort_values("offense_consensus_rank", na_position="last")
base.to_csv(OUT_PATH, index=False)
print(f"Saved FOX offense -> {OUT_PATH}  ({len(base)} teams)")
display(base.head(5))

Saved FOX offense -> ../data/offense_fox_2025.csv  (32 teams)


/var/folders/6l/18s2w2wn2bd93c3r28h5qs_h0000gn/T/ipykernel_76794/2637429816.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tbl))[0]
/var/folders/6l/18s2w2wn2bd93c3r28h5qs_h0000gn/T/ipykernel_76794/2637429816.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tbl))[0]
/var/folders/6l/18s2w2wn2bd93c3r28h5qs_h0000gn/T/ipykernel_76794/2637429816.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tbl))[0]
/var/folders/6l/18s2w2wn2bd93c3r28h5qs_h0000gn/T/ipykernel_76794/2637429816.py:75: FutureWarning: Passing literal html to 'read_html' is deprecat

,team_key,yds_g,off_td,pass_yds_g,pass_td,rush_yds_g,rush_td,pts_g,total_td,rz_td_pct,...,pts_g_rank,total_td_rank,rz_td_pct_rank,z_yds_g,z_pass_yds_g,z_rush_yds_g,z_pts_g,z_rz_td_pct,offense_consensus_score,offense_consensus_rank
3,Buffalo,420.0,12,257.0,5,163.0,7,34.0,12,66.7,...,4.0,3.0,10.0,1.975128,1.257100,1.932948,1.887878,0.467169,1.504045,1.0
10,Detroit,394.3,13,245.3,7,149.0,6,34.3,13,68.8,...,2.0,2.0,9.0,1.472981,0.959121,1.409741,1.938687,0.572059,1.270518,2.0
13,Indianapolis,418.7,10,265.3,3,153.3,6,34.3,10,43.8,...,2.0,5.0,26.0,1.949727,1.468488,1.570440,1.938687,-0.676638,1.250141,3.0
8,Dallas,393.7,7,268.7,3,125.0,4,24.7,7,60.0,...,10.0,16.0,16.0,1.461258,1.555080,0.512814,0.312794,0.132518,0.794893,4.0
31,Washington,354.0,10,196.7,4,157.3,5,26.7,10,75.0,...,7.0,5.0,5.0,0.685567,-0.278639,1.719928,0.651522,0.881736,0.732023,5.0


Best Offense

In [38]:
# === Build Offense Power Ranks (TDs > Yards in pass & rush) ===
import pandas as pd
import numpy as np
from pathlib import Path

YEAR = 2025
DATA = Path("../data")
IN_PATH  = DATA / f"offense_fox_{YEAR}.csv"
OUT_PATH = DATA / f"offense_power_{YEAR}.csv"

df = pd.read_csv(IN_PATH)

# ---- helpers ----
def zscore(s):
    s = pd.to_numeric(s, errors="coerce")
    mu = s.mean()
    sd = s.std(ddof=0)
    if sd == 0 or pd.isna(sd):
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - mu) / sd

def rank_desc(s):
    # 1 = best
    return pd.to_numeric(s, errors="coerce").rank(method="min", ascending=False)

# Ensure expected columns exist (fill with NaN if missing so formulas don't break)
needed = [
    "team_key",
    "yds_g","off_td",
    "pass_yds_g","pass_td",
    "rush_yds_g","rush_td",
    "pts_g","total_td","rz_td_pct",
]
for c in needed:
    if c not in df.columns:
        df[c] = np.nan

# ---- Weights inside PASS and RUSH parts (TDs > Yards) ----
W_PASS_TD   = 0.60
W_PASS_YDS  = 0.40
W_RUSH_TD   = 0.60
W_RUSH_YDS  = 0.40

# ---- Component scores (z-averages; TDs weighted higher in pass & rush) ----
# overall production (yards + offensive TD)
df["score_total"] = zscore(df["yds_g"]) + zscore(df["off_td"])

# passing offense (weighted TDs & yards)
df["score_pass"]  = (W_PASS_YDS * zscore(df["pass_yds_g"])
                     + W_PASS_TD  * zscore(df["pass_td"]))

# rushing offense (weighted TDs & yards)
df["score_rush"]  = (W_RUSH_YDS * zscore(df["rush_yds_g"])
                     + W_RUSH_TD * zscore(df["rush_td"]))

# scoring & red-zone (unchanged)
df["score_points"]= zscore(df["pts_g"]) + zscore(df["total_td"])
df["score_rz"]    = zscore(df["rz_td_pct"])

# Re-standardize part scores to keep them comparable
for c in ["score_total","score_pass","score_rush","score_points","score_rz"]:
    df[c] = zscore(df[c])

# Ranks for each part (1 = best)
df["rank_total"]  = rank_desc(df["score_total"])
df["rank_pass"]   = rank_desc(df["score_pass"])
df["rank_rush"]   = rank_desc(df["score_rush"])
df["rank_points"] = rank_desc(df["score_points"])
df["rank_rz"]     = rank_desc(df["score_rz"])

# ---- Overall offense score (blend of parts) ----
# You can tweak these; they must sum to 1.0 once normalized below
weights = {
    "score_total":  0.20,
    "score_pass":   0.25,
    "score_rush":   0.20,
    "score_points": 0.25,
    "score_rz":     0.10,
}
w = pd.Series(weights)
present = [k for k in weights if k in df.columns]
w = w.loc[present] / w.loc[present].sum()

df["offense_power_score"] = sum(df[c] * w[c] for c in present)
df["offense_power_rank"]  = rank_desc(df["offense_power_score"])

# Output
keep_cols = [
    "team_key",
    # raw metrics (keep whichever exist)
    *[c for c in ["yds_g","off_td",
                  "pass_yds_g","pass_td",
                  "rush_yds_g","rush_td",
                  "pts_g","total_td","rz_td_pct"] if c in df.columns],
    # part scores & ranks
    "score_total","rank_total",
    "score_pass","rank_pass",
    "score_rush","rank_rush",
    "score_points","rank_points",
    "score_rz","rank_rz",
    # overall
    "offense_power_score","offense_power_rank",
]
out = df[keep_cols].sort_values("offense_power_rank", na_position="last").reset_index(drop=True)

out.to_csv(OUT_PATH, index=False)
print(f"Saved offense power ranks -> {OUT_PATH}  ({len(out)} teams)")
display(out.head(5))

Saved offense power ranks -> ../data/offense_power_2025.csv  (32 teams)


,team_key,yds_g,off_td,pass_yds_g,pass_td,rush_yds_g,rush_td,pts_g,total_td,rz_td_pct,...,score_pass,rank_pass,score_rush,rank_rush,score_points,rank_points,score_rz,rank_rz,offense_power_score,offense_power_rank
0,Detroit,394.3,13,245.3,7,149.0,6,34.3,13,68.8,...,1.606059,2.0,1.797657,3.0,1.992737,2.0,0.572059,9.0,1.730358,1.0
1,Buffalo,420.0,12,257.0,5,163.0,7,34.0,12,66.7,...,0.849993,6.0,2.379014,1.0,1.778502,3.0,0.467169,10.0,1.609358,2.0
2,Baltimore,330.7,14,208.0,9,122.7,4,37.0,14,54.5,...,2.031552,1.0,0.662294,8.0,2.413703,1.0,-0.142195,17.0,1.538897,3.0
3,Indianapolis,418.7,10,265.3,3,153.3,6,34.3,10,43.8,...,0.050022,16.0,1.866838,2.0,1.427556,4.0,-0.676638,26.0,1.012946,4.0
4,Seattle,332.3,12,221.5,5,110.8,5,27.8,12,64.3,...,0.391580,11.0,0.826956,6.0,1.244448,5.0,0.347294,12.0,0.833398,5.0


> Defense

In [39]:
# === FOX Sports Defense Tables (collision-safe) -> defense_fox_{YEAR}.csv ===
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from pathlib import Path

YEAR = 2025
DATA = Path("../data"); DATA.mkdir(parents=True, exist_ok=True)
FOX_URL = "https://www.foxsports.com/articles/nfl/2025-nfl-defense-rankings-team-pass-and-rush-stats"
OUT_PATH = DATA / f"defense_fox_{YEAR}.csv"

# ---- Team mapping: FOX labels -> your city-style keys ----
TEAM_KEY = {
    "Arizona Cardinals":"Arizona","Arizona":"Arizona","Cardinals":"Arizona",
    "Atlanta Falcons":"Atlanta","Atlanta":"Atlanta","Falcons":"Atlanta",
    "Carolina Panthers":"Carolina","Carolina":"Carolina","Panthers":"Carolina",
    "Chicago Bears":"Chicago","Chicago":"Chicago","Bears":"Chicago",
    "Dallas Cowboys":"Dallas","Dallas":"Dallas","Cowboys":"Dallas",
    "Detroit Lions":"Detroit","Detroit":"Detroit","Lions":"Detroit",
    "Green Bay Packers":"Green Bay","Green Bay":"Green Bay","Packers":"Green Bay",
    "Los Angeles Rams":"LA Rams","L.A. Rams":"LA Rams","LA Rams":"LA Rams","Rams":"LA Rams",
    "Minnesota Vikings":"Minnesota","Minnesota":"Minnesota","Vikings":"Minnesota",
    "New Orleans Saints":"New Orleans","New Orleans":"New Orleans","Saints":"New Orleans",
    "New York Giants":"NY Giants","NY Giants":"NY Giants","Giants":"NY Giants",
    "Philadelphia Eagles":"Philadelphia","Philadelphia":"Philadelphia","Eagles":"Philadelphia",
    "San Francisco 49ers":"San Francisco","San Francisco":"San Francisco","49ers":"San Francisco",
    "Seattle Seahawks":"Seattle","Seattle":"Seattle","Seahawks":"Seattle",
    "Tampa Bay Buccaneers":"Tampa Bay","Tampa Bay":"Tampa Bay","Buccaneers":"Tampa Bay","Bucs":"Tampa Bay",
    "Washington Commanders":"Washington","Washington":"Washington","Commanders":"Washington",

    "Baltimore Ravens":"Baltimore","Baltimore":"Baltimore","Ravens":"Baltimore",
    "Buffalo Bills":"Buffalo","Buffalo":"Buffalo","Bills":"Buffalo",
    "Cincinnati Bengals":"Cincinnati","Cincinnati":"Cincinnati","Bengals":"Cincinnati",
    "Cleveland Browns":"Cleveland","Cleveland":"Cleveland","Browns":"Cleveland",
    "Denver Broncos":"Denver","Denver":"Denver","Broncos":"Denver",
    "Houston Texans":"Houston","Houston":"Houston","Texans":"Houston",
    "Indianapolis Colts":"Indianapolis","Indianapolis":"Indianapolis","Colts":"Indianapolis",
    "Jacksonville Jaguars":"Jacksonville","Jacksonville":"Jacksonville","Jaguars":"Jacksonville",
    "Kansas City Chiefs":"Kansas City","Kansas City":"Kansas City","Chiefs":"Kansas City",
    "Las Vegas Raiders":"Las Vegas","Las Vegas":"Las Vegas","Raiders":"Las Vegas",
    "Los Angeles Chargers":"LA Chargers","L.A. Chargers":"LA Chargers","LA Chargers":"LA Chargers","Chargers":"LA Chargers",
    "Miami Dolphins":"Miami","Miami":"Miami","Dolphins":"Miami",
    "New England Patriots":"New England","New England":"New England","Patriots":"New England",
    "New York Jets":"NY Jets","NY Jets":"NY Jets","Jets":"NY Jets",
    "Pittsburgh Steelers":"Pittsburgh","Pittsburgh":"Pittsburgh","Steelers":"Pittsburgh",
    "Tennessee Titans":"Tennessee","Tennessee":"Tennessee","Titans":"Tennessee",
}
norm = lambda s: re.sub(r"\s+"," ", str(s or "")).strip()
to_key = lambda s: TEAM_KEY.get(norm(s), norm(s))

# ---- fetch ----
sess = requests.Session()
sess.headers.update({
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/120.0.0.0 Safari/537.36")
})
r = sess.get(FOX_URL, timeout=25); r.raise_for_status()
soup = BeautifulSoup(r.text, "lxml")

def table_after_heading(heading_regex: str):
    h = soup.find(lambda tag: tag.name in {"h1","h2","h3","h4"} and re.search(heading_regex, tag.get_text(" "), re.I))
    return None if not h else h.find_next("table")

def parse_named_table(heading_regex: str, rename_map: dict, value_cols: list):
    """
    Return ONLY team_key + the requested value_cols (coerced numeric).
    Avoids '#'/rank collisions across merges.
    """
    tbl = table_after_heading(heading_regex)
    if tbl is None:
        return pd.DataFrame(columns=["team_key"] + value_cols)
    df = pd.read_html(str(tbl))[0]
    df.columns = [norm(c).lower() for c in df.columns]

    # Map headers to targets via regex keys
    col_map = {}
    for patt, target in rename_map.items():
        hit = next((c for c in df.columns if re.search(patt, c, re.I)), None)
        if hit:
            col_map[hit] = target
    df = df.rename(columns=col_map)

    # Find team column if not already set
    if "team" not in df.columns:
        cand = next((c for c in df.columns if "team" in c or "club" in c), None)
        if cand: df = df.rename(columns={cand:"team"})
    if "team" not in df.columns:
        return pd.DataFrame(columns=["team_key"] + value_cols)

    df["team_key"] = df["team"].map(to_key)
    df = df[df["team_key"].isin(set(TEAM_KEY.values()))]

    keep_vals = [c for c in value_cols if c in df.columns]
    out = df[["team_key"] + keep_vals].copy()

    # Coerce numeric
    for vc in keep_vals:
        out[vc] = (
            out[vc].astype(str)
                  .str.replace(",","", regex=False)
                  .str.replace("%","", regex=False)
                  .str.replace("T","", regex=False)
        )
        out[vc] = pd.to_numeric(out[vc], errors="coerce")
    return out

# ---- Parse DEFENSE sections (exact headings may vary slightly; regex is forgiving) ----
# For defense, lower is better; we will flip z-scores later so higher = better defense.
total_def = parse_named_table(
    r"Total Defense",
    rename_map={r"team":"team", r"defense.*yards.*avg|yards.*allowed.*avg": "yds_allowed_g",
                r"defense.*td|tds.*allowed": "def_td_allowed"},
    value_cols=["yds_allowed_g","def_td_allowed"]
)
pass_def = parse_named_table(
    r"Passing Defense",
    rename_map={r"team":"team", r"pass.*yards.*avg|pass.*allowed.*avg": "pass_yds_allowed_g",
                r"pass.*td.*allowed|passing.*td.*allowed": "pass_td_allowed"},
    value_cols=["pass_yds_allowed_g","pass_td_allowed"]
)
rush_def = parse_named_table(
    r"Rushing Defense",
    rename_map={r"team":"team", r"rush.*yards.*avg|rush.*allowed.*avg": "rush_yds_allowed_g",
                r"rush.*td.*allowed|rushing.*td.*allowed": "rush_td_allowed"},
    value_cols=["rush_yds_allowed_g","rush_td_allowed"]
)
score_def = parse_named_table(
    r"Scoring Defense|Points Allowed",
    rename_map={r"team":"team", r"points.*allowed.*avg|points.*per.*game|ppg": "pts_allowed_g"},
    value_cols=["pts_allowed_g"]
)
rz_def = parse_named_table(
    r"Red Zone Defense",
    rename_map={r"team":"team", r"red.*zone.*td.*rate|red.*zone.*percentage": "rz_td_allowed_pct"},
    value_cols=["rz_td_allowed_pct"]
)

# Optional: takeaways if FOX lists them in a table nearby (best-effort)
take_def = parse_named_table(
    r"Takeaways|Turnovers.*Gained|Defensive Takeaways",
    rename_map={r"team":"team", r"takeaways|turnovers.*gained|def\W*tk?": "takeaways"},
    value_cols=["takeaways"]
)

# ---- Combine safely ----
base = pd.DataFrame({"team_key": sorted(set(TEAM_KEY.values()))})
for d in [total_def, pass_def, rush_def, score_def, rz_def, take_def]:
    if not d.empty:
        base = base.merge(d, on="team_key", how="left")

# ---- Build defense scores (flip: lower allowed is better -> multiply by -1 before z) ----
def zscore_flip_lower_is_better(s):
    s = pd.to_numeric(s, errors="coerce")
    if s.std(ddof=0) == 0 or s.std(ddof=0) != s.std(ddof=0):  # NaN-safe
        return pd.Series(np.zeros(len(s)), index=s.index)
    # flip: lower allowed -> higher score
    return -(s - s.mean()) / s.std(ddof=0)

# Component scores
base["score_total_def"] = zscore_flip_lower_is_better(base.get("yds_allowed_g"))
if "def_td_allowed" in base:
    base["score_total_def"] += zscore_flip_lower_is_better(base["def_td_allowed"])

base["score_pass_def"]  = zscore_flip_lower_is_better(base.get("pass_yds_allowed_g"))
if "pass_td_allowed" in base:
    base["score_pass_def"] += zscore_flip_lower_is_better(base["pass_td_allowed"])

base["score_rush_def"]  = zscore_flip_lower_is_better(base.get("rush_yds_allowed_g"))
if "rush_td_allowed" in base:
    base["score_rush_def"] += zscore_flip_lower_is_better(base["rush_td_allowed"])

base["score_points_def"]= zscore_flip_lower_is_better(base.get("pts_allowed_g"))
if "takeaways" in base:
    # more takeaways is better (no flip)
    # standard z for takeaways, then add a small positive weight
    take = pd.to_numeric(base["takeaways"], errors="coerce")
    if take.notna().any():
        t_z = (take - take.mean()) / take.std(ddof=0)
        base["score_points_def"] += 0.3 * t_z.fillna(0)

base["score_rz_def"]    = zscore_flip_lower_is_better(base.get("rz_td_allowed_pct"))

# Re-standardize each score block so they’re on a comparable scale
def renorm(s):
    s = pd.to_numeric(s, errors="coerce")
    mu, sd = s.mean(), s.std(ddof=0)
    if sd == 0 or pd.isna(sd):
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - mu) / sd
for c in ["score_total_def","score_pass_def","score_rush_def","score_points_def","score_rz_def"]:
    base[c] = renorm(base[c])

# ---- Consensus defense score (higher = better defense). Keep rank optional. ----
weights = {
    "score_pass_def":   0.35,
    "score_rush_def":   0.35,
    "score_points_def": 0.20,
    "score_rz_def":     0.10,
}
w = pd.Series(weights)
present = [k for k in weights if k in base.columns]
w = w.loc[present] / w.loc[present].sum()

base["defense_consensus_score"] = sum(base[c] * w[c] for c in present)
base["defense_consensus_rank"]  = base["defense_consensus_score"].rank(method="min", ascending=False)

# ---- Save ----
base = base.sort_values("defense_consensus_rank", na_position="last")
base.to_csv(OUT_PATH, index=False)
print(f"Saved FOX defense -> {OUT_PATH}  ({len(base)} teams)")
display(base.head(5))

Saved FOX defense -> ../data/defense_fox_2025.csv  (32 teams)


/var/folders/6l/18s2w2wn2bd93c3r28h5qs_h0000gn/T/ipykernel_76794/269103675.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tbl))[0]
/var/folders/6l/18s2w2wn2bd93c3r28h5qs_h0000gn/T/ipykernel_76794/269103675.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tbl))[0]
/var/folders/6l/18s2w2wn2bd93c3r28h5qs_h0000gn/T/ipykernel_76794/269103675.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tbl))[0]
/var/folders/6l/18s2w2wn2bd93c3r28h5qs_h0000gn/T/ipykernel_76794/269103675.py:75: FutureWarning: Passing literal html to 'read_html' is deprecated a

,team_key,yds_allowed_g,pass_yds_allowed_g,pass_td_allowed,rush_yds_allowed_g,rush_td_allowed,pts_allowed_g,rz_td_allowed_pct,score_total_def,score_pass_def,score_rush_def,score_points_def,score_rz_def,defense_consensus_score,defense_consensus_rank
11,Green Bay,232.3,168.0,3,64.3,1,14.7,42.9,1.658568,1.147640,1.554022,1.501029,0.935582,1.339346,1.0
16,LA Chargers,276.7,182.0,2,94.7,2,16.7,25.0,0.803558,1.313573,0.578184,1.139199,2.069046,1.096859,2.0
1,Atlanta,227.3,131.0,3,96.3,2,19.7,60.0,1.754853,1.696152,0.545582,0.596454,-0.147224,0.889176,3.0
20,Minnesota,271.3,141.3,2,130.0,2,18.7,50.0,0.907546,1.916936,-0.141086,0.777369,0.485996,0.825621,4.0
27,San Francisco,265.7,162.0,4,103.7,1,16.3,55.6,1.015385,0.863110,0.751211,1.211565,0.131393,0.820465,5.0


In [61]:
# === Build Defense Power Ranks from FOX defense_fox_{YEAR}.csv ===
import pandas as pd
import numpy as np
from pathlib import Path

YEAR = 2025
DATA = Path("../data")
IN_PATH  = DATA / f"defense_fox_{YEAR}.csv"
OUT_PATH = DATA / f"defense_power_{YEAR}.csv"

df = pd.read_csv(IN_PATH)

# ---- helpers ----
def zscore(s):
    s = pd.to_numeric(s, errors="coerce")
    mu = s.mean()
    sd = s.std(ddof=0)
    if sd == 0 or pd.isna(sd):
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - mu) / sd

def zscore_flip_lower_is_better(s):
    # lower values are better (e.g., yards allowed) -> flip sign so higher = better
    s = pd.to_numeric(s, errors="coerce")
    mu = s.mean()
    sd = s.std(ddof=0)
    if sd == 0 or pd.isna(sd):
        return pd.Series(np.zeros(len(s)), index=s.index)
    return -(s - mu) / sd

def rank_desc(s):
    # 1 = best (higher score is better)
    return pd.to_numeric(s, errors="coerce").rank(method="min", ascending=False)

# ---- normalize column names & backfills ----
# Accept both naming variants for points allowed per game
if "pts_allowed_g" not in df.columns and "def_pts_allowed_g" in df.columns:
    df["pts_allowed_g"] = df["def_pts_allowed_g"]

# Ensure expected columns exist (we will backfill/NaN as needed)
needed = {
    "yds_allowed_g":      "Total yards allowed per game",
    "def_td_allowed":     "Total TDs allowed (we compute if missing)",
    "pass_yds_allowed_g": "Pass yards allowed per game",
    "pass_td_allowed":    "Pass TD allowed",
    "rush_yds_allowed_g": "Rush yards allowed per game",
    "rush_td_allowed":    "Rush TD allowed",
    "pts_allowed_g":      "Points allowed per game",
    "rz_td_allowed_pct":  "Red-zone TD allowed %",
    "takeaways":          "Defensive takeaways",
}
for c in needed:
    if c not in df.columns:
        df[c] = np.nan

# If total defensive TDs weren't provided, compute as pass + rush TDs
if df["def_td_allowed"].isna().all():
    df["def_td_allowed"] = df[["pass_td_allowed","rush_td_allowed"]].sum(axis=1, min_count=1)

# ---- component scores (flip where lower is better) ----
# Total defense (volume + TD prevention)
df["score_total_def"] = zscore_flip_lower_is_better(df["yds_allowed_g"]) \
                      + zscore_flip_lower_is_better(df["def_td_allowed"])

# Pass defense
df["score_pass_def"]  = zscore_flip_lower_is_better(df["pass_yds_allowed_g"]) \
                      + zscore_flip_lower_is_better(df["pass_td_allowed"])

# Rush defense
df["score_rush_def"]  = zscore_flip_lower_is_better(df["rush_yds_allowed_g"]) \
                      + zscore_flip_lower_is_better(df["rush_td_allowed"])

# Scoring defense (+ small bonus for takeaways if present)
df["score_points_def"]= zscore_flip_lower_is_better(df["pts_allowed_g"])
if df["takeaways"].notna().any():
    # more takeaways is better (no flip)
    take_z = zscore(df["takeaways"]).fillna(0)
    df["score_points_def"] = zscore(df["score_points_def"] + 0.3 * take_z)

# Red-zone defense (lower % allowed is better)
df["score_rz_def"]    = zscore_flip_lower_is_better(df["rz_td_allowed_pct"])

# Re-standardize part scores so they’re on the same scale
for c in ["score_total_def","score_pass_def","score_rush_def","score_points_def","score_rz_def"]:
    df[c] = zscore(df[c])

# Ranks for each part (1 = best defense)
df["rank_total_def"]  = rank_desc(df["score_total_def"])
df["rank_pass_def"]   = rank_desc(df["score_pass_def"])
df["rank_rush_def"]   = rank_desc(df["score_rush_def"])
df["rank_points_def"] = rank_desc(df["score_points_def"])
df["rank_rz_def"]     = rank_desc(df["score_rz_def"])

# ---- overall defense score (weighted blend of parts) ----
# Adjust weights as you like; they must sum to 1.0
weights = {
    "score_pass_def":   0.35,
    "score_rush_def":   0.35,
    "score_points_def": 0.20,
    "score_rz_def":     0.10,
}
w = pd.Series(weights)
present = [k for k in weights if k in df.columns]
w = w.loc[present] / w.loc[present].sum()

df["defense_power_score"] = sum(df[c] * w[c] for c in present)
df["defense_power_rank"]  = rank_desc(df["defense_power_score"])

# Nicely ordered output
keep_cols = [
    "team_key",
    # raw metrics (keep the ones that exist)
    *[c for c in ["yds_allowed_g","def_td_allowed",
                  "pass_yds_allowed_g","pass_td_allowed",
                  "rush_yds_allowed_g","rush_td_allowed",
                  "pts_allowed_g","rz_td_allowed_pct","takeaways"]
      if c in df.columns],
    # part scores & ranks
    "score_total_def","rank_total_def",
    "score_pass_def","rank_pass_def",
    "score_rush_def","rank_rush_def",
    "score_points_def","rank_points_def",
    "score_rz_def","rank_rz_def",
    # overall
    "defense_power_score","defense_power_rank",
]
out = df[keep_cols].sort_values("defense_power_rank", na_position="last").reset_index(drop=True)

out.to_csv(OUT_PATH, index=False)
print(f"Saved defense power ranks -> {OUT_PATH}  ({len(out)} teams)")
display(out.head(5))

Saved defense power ranks -> ../data/defense_power_2025.csv  (32 teams)


,team_key,yds_allowed_g,def_td_allowed,pass_yds_allowed_g,pass_td_allowed,rush_yds_allowed_g,rush_td_allowed,pts_allowed_g,rz_td_allowed_pct,takeaways,...,score_pass_def,rank_pass_def,score_rush_def,rank_rush_def,score_points_def,rank_points_def,score_rz_def,rank_rz_def,defense_power_score,defense_power_rank
0,Green Bay,232.3,4,168.0,3,64.3,1,14.7,42.9,NaN,...,1.147640,4.0,1.554022,2.0,1.501029,1.0,0.935582,3.0,1.339346,1.0
1,LA Chargers,276.7,4,182.0,2,94.7,2,16.7,25.0,NaN,...,1.313573,3.0,0.578184,11.0,1.139199,3.0,2.069046,1.0,1.096859,2.0
2,Atlanta,227.3,5,131.0,3,96.3,2,19.7,60.0,NaN,...,1.696152,2.0,0.545582,13.0,0.596454,13.0,-0.147224,19.0,0.889176,3.0
3,Minnesota,271.3,4,141.3,2,130.0,2,18.7,50.0,NaN,...,1.916936,1.0,-0.141086,17.0,0.777369,9.0,0.485996,11.0,0.825621,4.0
4,San Francisco,265.7,5,162.0,4,103.7,1,16.3,55.6,NaN,...,0.863110,9.0,0.751211,9.0,1.211565,2.0,0.131393,15.0,0.820465,5.0


# Weighted rankings

In [73]:
# === Season-to-date consensus team strength (32 rows) =======================
import pandas as pd
import numpy as np
from pathlib import Path

YEAR = 2025
DATA = Path("../data")

# Your weights (can tweak freely)
WEIGHTS = {
    "wpct": 0.20,              # most important
    "sos":  0.05,              # strength of schedule (opponents' wpct played to date)
    "edge_off_overall": 0.15,  # avg offense-vs-opp-defense edge across games played
    "edge_def_overall": 0.15,  # avg defense-vs-opp-offense edge across games played
    "power": 0.45,             # consensus media rank (inverted so lower rank -> higher score)
}

# ------------------------- helpers -------------------------
def inv_rank(r):
    """Turn rank (1=best..32=worst) into 'score' (32..1)."""
    r = pd.to_numeric(r, errors="coerce")
    return 33 - r

def minmax01(s):
    """Min-max scale to [0,1] with safe fallback for constant/all-NaN."""
    s = pd.to_numeric(s, errors="coerce")
    lo, hi = s.min(skipna=True), s.max(skipna=True)
    if pd.isna(lo) or pd.isna(hi) or lo == hi:
        return pd.Series(np.full(len(s), 0.5), index=s.index)
    return (s - lo) / (hi - lo)

def safe_read(path):
    try:
        return pd.read_csv(path)
    except Exception:
        return pd.DataFrame()

# ------------------------- load inputs -------------------------
results = safe_read(DATA / f"results_teamweek_{YEAR}.csv")        # team-week long, has 'win' boolean
off     = safe_read(DATA / f"offense_power_{YEAR}.csv")           # team_key, rank_pass, rank_rush (names may vary)
def_    = safe_read(DATA / f"defense_power_{YEAR}.csv")           # team_key, rank_pass_def, rank_rush_def
cons    = safe_read(DATA / f"consensus_rank_{YEAR}.csv")          # team_key + some rank col

# normalize team column names to 'team'
for df in (off, def_, cons):
    if "team" not in df.columns and "team_key" in df.columns:
        df.rename(columns={"team_key":"team"}, inplace=True)

# pick columns flexibly
def pick_col(df, keywords, must=None, default=None):
    cols = [c for c in df.columns if all(k in c.lower() for k in keywords)]
    if must:
        cols = [c for c in cols if must in c.lower()]
    if cols:
        cols.sort(key=len)
        return cols[0]
    return default

col_off_pass  = pick_col(off, ["pass"], must="rank", default="rank_pass")
col_off_rush  = pick_col(off, ["rush"], must="rank", default="rank_rush")
col_def_ppass = pick_col(def_,["pass","def"], must="rank", default="rank_pass_def")
col_def_prush = pick_col(def_,["rush","def"], must="rank", default="rank_rush_def")
col_cons_rank = pick_col(cons,["consensus","rank"], default=pick_col(cons,["avg","power","rank"], default=None))
if col_cons_rank is None:
    # If your consensus file just has 'rank' use that
    col_cons_rank = "rank" if "rank" in cons.columns else cons.columns[-1]

# ------------------------- W/L pct & SOS -------------------------
# Use only completed games
if "completed" in results.columns:
    res = results[results["completed"] == True].copy()
else:
    res = results.copy()

# Clean columns
need_cols = ["team","opponent","win"]
missing = [c for c in need_cols if c not in res.columns]
if missing:
    raise KeyError(f"results_teamweek is missing columns: {missing}")

# W/L per team
wl = (res.groupby("team", as_index=False)
        .agg(wins=("win","sum"),
             games=("win","size")))
wl["losses"] = wl["games"] - wl["wins"]
wl["wpct"]   = np.where(wl["games"]>0, wl["wins"]/wl["games"], np.nan)

# Opponents' current wpct (SOS to date = mean opponent wpct for games played)
opp_wpct = wl.set_index("team")["wpct"].to_dict()
res["opp_wpct_now"] = res["opponent"].map(opp_wpct)
sos = (res.groupby("team", as_index=False)
         .agg(sos=("opp_wpct_now","mean")))

# ------------------------- matchup edges (season-to-date averages) ----------
# Prepare offense & defense rank lookup (convert to scores where higher is better)
off_lu = off[["team", col_off_pass, col_off_rush]].copy()
off_lu["pass_off_sc"] = inv_rank(off_lu[col_off_pass])
off_lu["rush_off_sc"] = inv_rank(off_lu[col_off_rush])
off_lu = off_lu[["team","pass_off_sc","rush_off_sc"]]

def_lu = def_[["team", col_def_ppass, col_def_prush]].copy()
def_lu["pass_def_sc"] = inv_rank(def_lu[col_def_ppass])
def_lu["rush_def_sc"] = inv_rank(def_lu[col_def_prush])
def_lu = def_lu[["team","pass_def_sc","rush_def_sc"]]

# Join team offense + team defense + opponent offense + opponent defense
tmp = res.merge(off_lu, on="team", how="left")
tmp = tmp.merge(def_lu, on="team", how="left")  # team's own defense
tmp = tmp.merge(def_lu.rename(columns={
                    "team":"opponent",
                    "pass_def_sc":"opp_pass_def_sc",
                    "rush_def_sc":"opp_rush_def_sc"
                }),
                on="opponent", how="left")
tmp = tmp.merge(off_lu.rename(columns={
                    "team":"opponent",
                    "pass_off_sc":"opp_pass_off_sc",
                    "rush_off_sc":"opp_rush_off_sc"
                }),
                on="opponent", how="left")

# Offense edge vs opponent defense; Defense edge vs opponent offense
tmp["edge_off_game"] = (tmp["pass_off_sc"] + tmp["rush_off_sc"])/2.0 - (tmp["opp_pass_def_sc"] + tmp["opp_rush_def_sc"])/2.0
tmp["edge_def_game"] = (tmp["pass_def_sc"] + tmp["rush_def_sc"])/2.0 - (tmp["opp_pass_off_sc"] + tmp["opp_rush_off_sc"])/2.0

edges = (tmp.groupby("team", as_index=False)
           .agg(edge_off_overall=("edge_off_game","mean"),
                edge_def_overall=("edge_def_game","mean")))

# ------------------------- consensus power -------------------------
power = cons[["team", col_cons_rank]].rename(columns={col_cons_rank:"consensus_rank"}).copy()
power["power_score"] = minmax01(inv_rank(power["consensus_rank"]))  # higher is better

# ------------------------- assemble master 32-row table ---------------------
teams = pd.DataFrame({"team": sorted(set(off["team"]) | set(def_["team"]) | set(wl["team"]))})

out = (teams
    .merge(wl[["team","wins","losses","games","wpct"]], on="team", how="left")
    .merge(sos, on="team", how="left")
    .merge(edges, on="team", how="left")
    .merge(power[["team","consensus_rank","power_score"]], on="team", how="left"))

# Scale everything to 0..1 (except consensus already scaled)
out["wpct_s"]  = out["wpct"]           # already 0..1
out["sos_s"]   = out["sos"]            # 0..1-ish (can be NaN early)
out["edge_off_s"] = minmax01(out["edge_off_overall"])
out["edge_def_s"] = minmax01(out["edge_def_overall"])
out["power_s"] = out["power_score"].fillna(0.5)

# Fill missing with neutral
for c in ["wpct_s","sos_s","edge_off_s","edge_def_s","power_s"]:
    out[c] = out[c].fillna(0.5)

# Weighted total per your weights
out["total_score"] = (
    WEIGHTS["wpct"]            * out["wpct_s"] +
    WEIGHTS["sos"]             * out["sos_s"] +
    WEIGHTS["edge_off_overall"]* out["edge_off_s"] +
    WEIGHTS["edge_def_overall"]* out["edge_def_s"] +
    WEIGHTS["power"]           * out["power_s"]
)

# Overall rank (1 = strongest)
out["overall_rank"] = out["total_score"].rank(method="min", ascending=False)

# Tidy columns / save
keep = [
    "team","wins","losses","games","wpct","sos",
    "edge_off_overall","edge_def_overall",
    "consensus_rank","total_score","overall_rank"
]
out_final = out[keep].sort_values("overall_rank").reset_index(drop=True)

SAVE_PATH = DATA / f"team_consensus_strength_{YEAR}.csv"
out_final.to_csv(SAVE_PATH, index=False)
print(f"Saved team consensus strength -> {SAVE_PATH}  ({len(out_final)} teams)")
display(out_final.head(12))

Saved team consensus strength -> ../data/team_consensus_strength_2025.csv  (32 teams)


,team,wins,losses,games,wpct,sos,edge_off_overall,edge_def_overall,consensus_rank,total_score,overall_rank
0,Buffalo,3,0,3,1.000000,0.111111,22.333333,-2.166667,2.00,0.869004,1.0
1,LA Chargers,3,0,3,1.000000,0.333333,-0.500000,11.500000,3.50,0.830791,2.0
2,Tampa Bay,3,0,3,1.000000,0.111111,1.500000,12.166667,4.50,0.815103,3.0
3,Philadelphia,3,0,3,1.000000,0.444444,-0.500000,-2.833333,1.00,0.814028,4.0
4,Indianapolis,3,0,3,1.000000,0.111111,11.666667,10.333333,10.75,0.753300,5.0
5,LA Rams,2,1,3,0.666667,0.333333,9.000000,15.333333,9.25,0.730488,6.0
6,Detroit,2,1,3,0.666667,0.444444,18.333333,-11.333333,4.50,0.730113,7.0
7,Green Bay,2,1,3,0.666667,0.555556,-1.500000,10.166667,6.25,0.725854,8.0
8,San Francisco,3,0,3,1.000000,0.416667,0.333333,7.333333,10.25,0.722103,9.0
9,Washington,2,1,3,0.666667,0.333333,2.000000,4.833333,8.75,0.669044,10.0


# Projected strength of schedule

In [74]:
# === Projected Strength of Schedule (power + matchup) ======================
import pandas as pd
import numpy as np
from pathlib import Path

YEAR = 2025
WEEK  = 4     # <-- change this each week (same as your WEEK_OVERRIDE)
DATA  = Path("../data")

# ------------------ Load required inputs ------------------
sched  = pd.read_csv(DATA / f"results_teamweek_{YEAR}.csv")     # full schedule (team-week long)
off    = pd.read_csv(DATA / f"offense_power_{YEAR}.csv")        # has rank_pass, rank_rush (names may vary)
def_   = pd.read_csv(DATA / f"defense_power_{YEAR}.csv")        # has rank_pass_def, rank_rush_def
cons   = pd.read_csv(DATA / f"consensus_rank_{YEAR}.csv")       # has team_key + consensus/avg rank

# Normalize team col to 'team'
for df in (off, def_, cons):
    if "team" not in df.columns and "team_key" in df.columns:
        df.rename(columns={"team_key":"team"}, inplace=True)

# Flexible column pickers
def pick_col(df, includes, must_contain=None, default=None):
    cols = [c for c in df.columns if all(k in c.lower() for k in includes)]
    if must_contain:
        cols = [c for c in cols if must_contain in c.lower()]
    if cols:
        cols.sort(key=len)
        return cols[0]
    return default

col_off_pass  = pick_col(off,  ["pass"], must_contain="rank", default="rank_pass")
col_off_rush  = pick_col(off,  ["rush"], must_contain="rank", default="rank_rush")
col_def_ppass = pick_col(def_, ["pass","def"], must_contain="rank", default="rank_pass_def")
col_def_prush = pick_col(def_, ["rush","def"], must_contain="rank", default="rank_rush_def")
col_cons_rank = pick_col(cons, ["consensus","rank"], default=pick_col(cons,["avg","power","rank"], default=None))
if col_cons_rank is None:
    col_cons_rank = "rank" if "rank" in cons.columns else cons.columns[-1]

# --------------- Helpers: scores from ranks, z, etc. ---------------
# Rank: 1 (best) .. 32 (worst). We'll create:
#  - strength_score = (33 - rank)  -> higher = stronger team
#  - difficulty     = rank/32      -> higher = tougher opponent
def inv_rank(r):
    r = pd.to_numeric(r, errors="coerce")
    return 33 - r

def difficulty_from_rank(r):
    r = pd.to_numeric(r, errors="coerce")
    return r / 32.0

def z(s):
    s = pd.to_numeric(s, errors="coerce")
    m, sd = s.mean(), s.std(ddof=0)
    if sd == 0 or pd.isna(sd): 
        return pd.Series(np.zeros(len(s)), index=s.index)
    return (s - m)/sd

# --------------- Build difficulty lookups ----------------
# Power difficulty (opponent difficulty this week)
power_lu = cons[["team", col_cons_rank]].rename(columns={col_cons_rank:"consensus_rank"}).copy()
power_lu["opp_difficulty_power"] = difficulty_from_rank(power_lu["consensus_rank"])
power_lu = power_lu[["team","opp_difficulty_power"]]

# Offense/defense scores for matchup difficulty
off_lu = off[["team", col_off_pass, col_off_rush]].copy()
off_lu["pass_off_sc"] = inv_rank(off_lu[col_off_pass])
off_lu["rush_off_sc"] = inv_rank(off_lu[col_off_rush])
off_lu = off_lu[["team","pass_off_sc","rush_off_sc"]]

def_lu = def_[["team", col_def_ppass, col_def_prush]].copy()
def_lu["pass_def_sc"] = inv_rank(def_lu[col_def_ppass])
def_lu["rush_def_sc"] = inv_rank(def_lu[col_def_prush])
def_lu = def_lu[["team","pass_def_sc","rush_def_sc"]]

# --------------- Prepare schedule (future only) ---------------
# Keep one row per (team, week) going forward
sch = sched.copy()
sch = sch[["week","team","opponent","home_away"]].drop_duplicates()
sch = sch[sch["week"] >= WEEK].reset_index(drop=True)

# Attach THIS-WEEK opponent power difficulty
sch = sch.merge(power_lu.rename(columns={"team":"opponent"}),
                on="opponent", how="left")

# Attach team offense, opponent defense for matchup difficulty
sch = sch.merge(off_lu, on="team", how="left")
sch = sch.merge(def_lu.rename(columns={
                    "team":"opponent",
                    "pass_def_sc":"opp_pass_def_sc",
                    "rush_def_sc":"opp_rush_def_sc"
                }),
                on="opponent", how="left")

# Matchup difficulty for this game (higher = tougher for offense)
# We define 'matchup edge' = team_off - opp_def (higher = easier),
# so 'matchup difficulty' = negative z of that edge (higher = tougher).
sch["edge_pass"] = sch["pass_off_sc"] - sch["opp_pass_def_sc"]
sch["edge_rush"] = sch["rush_off_sc"] - sch["opp_rush_def_sc"]
sch["edge_mix"]  = 0.6*sch["edge_pass"] + 0.4*sch["edge_rush"]

# Normalize within slate to be stable, then invert to make "difficulty"
# Do it per-week so different slates are comparable.
sch["edge_mix_z"] = sch.groupby("week")["edge_mix"].transform(z)
sch["opp_difficulty_matchup"] = -sch["edge_mix_z"]   # higher => tougher matchup

# ----------------- Aggregate to FUTURE SoS from current week ---------------
# For every (team, week), compute the mean difficulty of ALL remaining games
def future_avg(df, col):
    return (df.groupby(["team","week"], as_index=False)
              .apply(lambda g: g[g["week"]>=g.name[1]][col].mean())
              .reset_index(drop=True))

# Power-based future SOS (mean of opponent power difficulty from week onward)
tmp_power = sch.groupby(["team","week"], as_index=False)["opp_difficulty_power"].mean()
tmp_power = tmp_power.rename(columns={"opp_difficulty_power":"this_week_power"})
# Expand to remaining-average per (team, week)
rem_power = []
for t, g in sch.groupby("team"):
    g = g.sort_values("week")
    # cumulative remaining mean from bottom
    vals = g["opp_difficulty_power"].values
    wks  = g["week"].values
    rem_mean = np.flip(np.cumsum(np.flip(vals)) / np.arange(1, len(vals)+1))
    rem_power.append(pd.DataFrame({"team":t, "week":wks, "future_sos_power":rem_mean}))
rem_power = pd.concat(rem_power, ignore_index=True)

# Matchup-based future SOS (mean of matchup difficulty from week onward)
rem_match = []
for t, g in sch.groupby("team"):
    g = g.sort_values("week")
    vals = g["opp_difficulty_matchup"].values
    wks  = g["week"].values
    rem_mean = np.flip(np.cumsum(np.flip(vals)) / np.arange(1, len(vals)+1))
    rem_match.append(pd.DataFrame({"team":t, "week":wks, "future_sos_matchup":rem_mean}))
rem_match = pd.concat(rem_match, ignore_index=True)

# Join the two future SOS views and build blended projection
proj = (rem_power
        .merge(rem_match, on=["team","week"], how="outer"))

# Blend: power carries more signal early, matchup adds shape
W_POWER, W_MATCHUP = 0.60, 0.40
proj["future_sos_proj"] = (W_POWER*proj["future_sos_power"].astype(float)
                           + W_MATCHUP*proj["future_sos_matchup"].astype(float))

# Also carry this-week's opponent power difficulty (handy for current slate features)
this_week_power = sch[["team","week","opp_difficulty_power"]].copy()
this_week_power = this_week_power.rename(columns={"opp_difficulty_power":"opp_difficulty_power_this_week"})
this_week_power = this_week_power.drop_duplicates(subset=["team","week"])

proj = proj.merge(this_week_power, on=["team","week"], how="left")

# ----------------- Save / handoff to survivor code -----------------
OUT_PATH = DATA / f"projected_sos_{YEAR}.csv"
proj.to_csv(OUT_PATH, index=False)
print(f"Saved projected SOS (power + matchup) -> {OUT_PATH}  ({len(proj)} rows)")
display(proj.sort_values(["week","future_sos_proj"]).head(12))

Saved projected SOS (power + matchup) -> ../data/projected_sos_2025.csv  (448 rows)


,team,week,future_sos_power,future_sos_matchup,future_sos_proj,opp_difficulty_power_this_week
140,Detroit,4,0.444754,-1.371417,-0.281714,0.765625
28,Baltimore,4,0.575893,-1.367079,-0.201296,0.289062
42,Buffalo,4,0.632254,-1.290009,-0.136651,0.992188
70,Chicago,4,0.454241,-0.682894,-0.000613,0.757812
238,LA Rams,4,0.464286,-0.575013,0.048566,0.335938
154,Green Bay,4,0.503906,-0.618093,0.055106,0.710938
392,Seattle,4,0.492746,-0.433785,0.122133,0.507812
224,LA Chargers,4,0.564174,-0.518916,0.130938,0.914062
112,Dallas,4,0.434710,-0.238355,0.165484,0.195312
182,Indianapolis,4,0.534598,-0.378797,0.169240,0.289062
